In [ ]:
#imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import urllib.parse
from glob import glob
import os
import ntpath 
import zipfile

#paths
PPMI_zippath='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018.zip'
PPMI_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/'

In [2]:
# unzipping the PPMI data folder
#zip_ref = zipfile.ZipFile(PPMI_path, 'r')
#zip_ref.extractall()
#zip_ref.close()

## Algorithm for reading all the csv files at once
#### 0. Set the right path folder (Study Enrollment, Medical, Scans...) 
#### 1. FOR LOOP on the .csv files
    #### IN FOR LOOP: 
    *read the csv file, and save it in a dataframe
    *save the number of rows (length)
    *save the number of colums
    * save the df
#### 2. Get a list (or array) containing (at least) each df for each csv file. 
#### 3. Explore the different df. 

## 1-Subject-Characteristics

In [11]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'

In [16]:
# saving all the csv files in a dataframe
import os
import glob
# glob.glob('*.csv') #find all the csv files in a pathname. 
os.chdir(folder_path)
csv_files = [i for i in glob.glob('*.csv')]
#med_files=glob(folder_path)
csv_files

['Patient_Status.csv',
 'Socio-Economics.csv',
 'Family_History__PD_.csv',
 'Screening___Demographics.csv',
 'Center-Subject_List.csv',
 'Randomization_table.csv']

In [57]:
# Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
dict_files={}
files_names=[]
for files in csv_files:
#for i, files in enumerate(csv_files): 
    df=pd.read_csv(files,engine='python')
    basepath, filename=ntpath.split(files)
    files_names.append(filename)
    dict_files[filename]=df
#list(dict_files.keys())

# Calculating the # of rows and features for each dataframe and storing them in a list of tuples
shape_df=[]
for i in range(0,len(dict_files.keys())):
    nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
    shape_df.append((nrows,ncols))
shape_df

[(2135, 8), (2060, 12), (2021, 28), (2144, 29), (2144, 2), (1809, 14)]

### Patient_Status - useless

In [56]:
PRODROMA=dict_files[files_names[0]]
PRODROMA.head()

,PATNO,RECRUITMENT_CAT,IMAGING_CAT,ENROLL_DATE,ENROLL_CAT,ENROLL_STATUS,DESCRP_CAT,STATUS_DATE
0,10362,PRODROMA,PRODROMA,NaN,NaN,Excluded,HYP,08/2013
1,10405,PRODROMA,PRODROMA,NaN,NaN,Excluded,HYP,09/2014
2,10496,PRODROMA,PRODROMA,NaN,NaN,Excluded,HYP,06/2013
3,10606,PRODROMA,no image,NaN,NaN,Declined,HYP,08/2013
4,10662,PRODROMA,PRODROMA,NaN,NaN,Excluded,HYP,06/2014


### Socio-Economics
#### -Very likely useless: could be only useful if there is a correlation between the handedness and the motors test results

In [45]:
SOCIOECO=dict_files[files_names[1]]
SOCIOECO.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,EDUCYRS,HANDED,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,408734501,V,10362,SC,SOCIOECO,08/2013,16.0,1.0,08/2013,2013-08-23 09:18:20.0,NaN,08/2013
1,605734901,V,10405,SC,SOCIOECO,09/2014,22.0,1.0,12/2016,2016-12-06 12:42:46.0,NaN,12/2016
2,404093001,V,10496,SC,SOCIOECO,06/2013,12.0,1.0,07/2013,2013-07-08 11:59:09.0,NaN,07/2013
3,409585901,S,10606,SC,SOCIOECO,08/2013,16.0,1.0,09/2013,2015-06-26 08:16:19.0,NaN,03/2015
4,448228901,V,10662,SC,SOCIOECO,06/2014,21.0,1.0,07/2014,2014-07-15 11:06:59.0,NaN,07/2014


### Family_History_PD - most likely useful 
#### Can be useful for detecting early PD

In [46]:
FAMHXPD=dict_files[files_names[2]]
FAMHXPD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,BIOMOM,BIOMOMPD,BIODAD,BIODADPD,...,MATAU,MATAUPD,PATAU,PATAUPD,KIDSNUM,KIDSPD,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226912501,V,3403,SC,FAMHXPD,06/2010,1,0.0,1.0,0.0,...,4.0,0.0,1.0,0.0,3.0,0.0,06/2010,2011-07-18 10:07:31.0,NaN,07/2010
1,229758101,V,3402,SC,FAMHXPD,06/2010,1,0.0,1.0,0.0,...,2.0,0.0,2.0,0.0,4.0,0.0,06/2010,2011-09-28 09:47:05.0,NaN,06/2010
2,226872001,V,3401,SC,FAMHXPD,06/2010,1,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,2.0,0.0,06/2010,2011-08-02 09:43:13.0,NaN,06/2010
3,233877701,V,3404,SC,FAMHXPD,06/2010,1,0.0,1.0,0.0,...,2.0,0.0,6.0,0.0,2.0,0.0,07/2010,2011-04-26 12:35:40.0,NaN,07/2010
4,232504901,V,3405,SC,FAMHXPD,07/2010,1,0.0,1.0,0.0,...,4.0,0.0,6.0,0.0,NaN,NaN,07/2010,2010-07-13 12:43:35.0,NaN,07/2010


### Screening__Demographics - useful

In [47]:
SCREEN=dict_files[files_names[3]]
SCREEN.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,SIGNCNST,CONSNTDT,APPRDX,CURRENT_APPRDX,P3GRP,...,PRJENRDT,REFERRAL,DECLINED,RSNDEC,EXCLUDED,RSNEXC,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,224392801,V,3400,CONSENT,SCREEN,1.0,06/2010,1.0,1.0,NaN,...,07/2010,60.0,NaN,NaN,NaN,NaN,06/2010,2010-12-17 10:58:57.0,NaN,07/2010
1,224394301,V,3401,CONSENT,SCREEN,1.0,06/2010,2.0,2.0,NaN,...,07/2010,1.0,NaN,NaN,NaN,NaN,06/2010,2010-07-20 08:09:56.0,NaN,07/2010
2,224398401,V,3402,CONSENT,SCREEN,1.0,06/2010,3.0,3.0,NaN,...,10/2011,1.0,NaN,NaN,NaN,NaN,06/2010,2011-09-27 12:12:25.0,NaN,07/2010
3,224400201,V,3403,CONSENT,SCREEN,1.0,06/2010,1.0,1.0,NaN,...,07/2010,1.0,NaN,NaN,NaN,NaN,06/2010,2010-07-20 08:35:45.0,NaN,07/2010
4,224718101,V,3404,CONSENT,SCREEN,1.0,06/2010,2.0,2.0,NaN,...,07/2010,1.0,NaN,NaN,NaN,NaN,06/2010,2010-07-20 09:00:48.0,NaN,07/2010


### Center_Subject_List - useless

In [51]:
CENTSUBJ=dict_files[files_names[4]]
CENTSUBJ.head()

,CNO,PATNO
0,34,3400
1,34,3401
2,34,3402
3,34,3403
4,34,3404


### Randomization_Table - useful
#### Age and gender to be extracted. 

In [54]:
RANDOM=dict_files[files_names[5]]
RANDOM.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,ENROLLDT,BIRTHDT,GENDER,CONSNTDT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,456176901,V,10874,RANDOM,RANDOM,NaN,09/2014,07/1941,1.0,07/2014,09/2014,2014-09-11 10:01:39.0,NaN,09/2014
1,416233501,V,12224,RANDOM,RANDOM,NaN,10/2013,05/1947,1.0,10/2013,10/2013,2013-10-30 13:04:22.0,NaN,10/2013
2,453909001,V,12499,RANDOM,RANDOM,NaN,08/2014,09/1949,2.0,10/2013,08/2014,2014-08-22 10:54:19.0,NaN,08/2014
3,429276101,V,12593,RANDOM,RANDOM,NaN,02/2014,01/1947,1.0,11/2013,02/2014,2014-02-27 06:38:23.0,NaN,02/2014
4,478302001,V,13039,RANDOM,RANDOM,NaN,11/2014,03/1947,2.0,09/2014,11/2014,2014-11-20 09:36:27.0,NaN,11/2014


## 2-Biospecimen

In [64]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/2-Biospecimen/'
# saving all the csv files in a dataframe
import os
import glob
# glob.glob('*.csv') #find all the csv files in a pathname. 
os.chdir(folder_path)
csv_files = [i for i in glob.glob('*.csv')]
csv_files

# Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
dict_files={}
files_names=[]
for files in csv_files:
#for i, files in enumerate(csv_files): 
    df=pd.read_csv(files,engine='python')
    basepath, filename=ntpath.split(files)
    files_names.append(filename)
    dict_files[filename]=df


# Calculating the # of rows and features for each dataframe and storing them in a list of tuples
shape_df=[]
for i in range(0,len(dict_files.keys())):
    nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
    shape_df.append((nrows,ncols))
print(shape_df)
print(list(dict_files.keys()))

# DF TO EXCEL
#from pandas import ExcelWriter

#writer = ExcelWriter('PythonExport.xlsx')
#yourdf.to_excel(writer,'Sheet5')
#writer.save()

[(6269, 13), (26, 22), (81701, 13), (2176, 13), (11214, 13), (137, 36), (2279, 13), (11452, 52), (11452, 60), (6022, 51), (168, 24), (5433, 13), (177835, 24), (2081, 14), (44227, 19)]
['Clinical_Labs.csv', 'Skin_Biopsy.csv', 'Current_Biospecimen_Analysis_Results.csv', 'Genetic_Testing_Results.csv', 'Whole_Blood_Sample_Collection.csv', 'iPSC_Blood_Sample.csv', 'Pilot_Biospecimen_Analysis_Results.csv', 'Laboratory_Procedures.csv', 'Laboratory_Procedures_with_Elapsed_Times.csv', 'Lumbar_Puncture_Sample_Collection.csv', 'IUSM_BIOSPECIMEN_CELL_CATALOG.csv', 'Deprecated_Biospecimen_Analysis_Results.csv', 'Blood_Chemistry___Hematology.csv', 'DNA_Sample_Collection.csv', 'IUSM_CATALOG.csv']


### Clinical_Labs - DNK - most likely useless

In [63]:
LAB=dict_files[files_names[0]]
LAB.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,BLDLAB,BLDSHPDT,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226845701,V,3400,SC,LAB,06/2010,1,06/2010,NaN,06/2010,2010-07-14 10:27:53.0,NaN,07/2010
1,226877401,V,3401,SC,LAB,06/2010,1,06/2010,NaN,06/2010,2010-07-14 10:28:44.0,NaN,06/2010
2,226894001,V,3402,SC,LAB,06/2010,1,06/2010,NaN,06/2010,2010-07-14 10:29:36.0,NaN,06/2010
3,226921901,V,3403,SC,LAB,06/2010,1,06/2010,NaN,06/2010,2010-07-14 10:31:53.0,NaN,07/2010
4,232503101,V,3405,SC,LAB,07/2010,1,07/2010,NaN,07/2010,2010-07-13 12:39:35.0,NaN,07/2010


### Skin_Biopsy - useful: Feat-bio

In [68]:
SKBIO=dict_files[files_names[1]]
SKBIO.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,SKBIOCMP,ANSTHADM,SKBIOLOC,SBIOLCCM,...,WOUNDCLS,WNDCLSCM,BIOCOLTM,BIOFRGTM,SHIPDT,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,455646501,V,3429,V08,SKBIO,08/2014,1,1,1,NaN,...,1,NaN,12:46:00,12:46:00,08/2014,NaN,09/2014,2014-09-08,NaN,09/2014
1,455667101,V,3430,V08,SKBIO,09/2014,1,1,1,NaN,...,1,NaN,11:49:00,11:49:00,09/2014,NaN,09/2014,2014-09-08,NaN,09/2014
2,455864201,V,3444,V07,SKBIO,08/2014,1,1,1,NaN,...,1,NaN,12:33:00,12:33:00,08/2014,NaN,09/2014,2014-09-09,NaN,09/2014
3,456430101,V,3431,V08,SKBIO,09/2014,1,1,3,NaN,...,1,NaN,14:21:00,14:21:00,09/2014,NaN,09/2014,2014-09-12,NaN,09/2014
4,456815901,V,3406,V10,SKBIO,09/2014,1,1,1,NaN,...,2,NaN,15:03:00,15:03:00,09/2014,NaN,09/2014,2014-09-15,NaN,09/2014


### Current_Biospecimen_Analysis_Results - DNK 
#### Most likel useless: the next table should be more useful. 

In [74]:
BIORESULTS=dict_files[files_names[2]]
BIORESULTS.head()

,PATNO,GENDER,DIAGNOSIS,CLINICAL_EVENT,TYPE,TESTNAME,TESTVALUE,UNITS,RUNDATE,PROJECTID,PI_NAME,PI_INSTITUTION,update_stamp
0,3400,Female,PD,SC,DNA,ApoE Genotype,e3/e3,NaN,2012-09-27,104,Andrew Singleton,National Institutes of Aging,2014-11-03 11:58:57.0
1,3401,Female,Control,SC,DNA,ApoE Genotype,e3/e3,NaN,2012-09-27,104,Andrew Singleton,National Institutes of Aging,2014-11-03 11:58:57.0
2,3402,Male,SWEDD,SC,DNA,ApoE Genotype,e3/e3,NaN,2012-09-27,104,Andrew Singleton,National Institutes of Aging,2014-11-03 11:58:57.0
3,3403,Male,PD,SC,DNA,ApoE Genotype,e3/e2,NaN,2012-09-27,104,Andrew Singleton,National Institutes of Aging,2014-11-03 11:58:57.0
4,3404,Female,Control,SC,DNA,ApoE Genotype,e4/e3,NaN,2012-09-27,104,Andrew Singleton,National Institutes of Aging,2014-11-03 11:58:57.0


### Genetic_Results_Tests - useful: Feat-bio

In [73]:
MUTRSLT=dict_files[files_names[3]]
MUTRSLT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,GENECAT,LRRKCD,MUTRSLT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,426136501,V,41386,GMU,MUTRSLT,01/2014,1,1.0,1,01/2014,2014-01-30 05:52:45.0,NaN,01/2014
1,445446201,V,50157,GMU,MUTRSLT,06/2014,1,1.0,1,06/2014,2014-06-24 06:25:08.0,NaN,06/2014
2,422444501,V,40819,GMU,MUTRSLT,01/2014,1,1.0,1,01/2014,2014-01-02 07:32:45.0,NaN,01/2014
3,423319501,V,40577,GMU,MUTRSLT,01/2014,1,1.0,1,01/2014,2014-01-09 10:58:27.0,NaN,01/2014
4,493347201,V,41564,GMU,MUTRSLT,01/2015,1,1.0,1,01/2015,2015-01-12 10:47:02.0,NaN,01/2015


### Whole_Blood_Sample_Collection - useless

In [331]:
WHOLEBLD=dict_files[files_names[4]]
WHOLEBLD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,SIGNCNST,SKBICNDT,INEX1,INEX2,INEX3,INEX4,INEX5,INEX6,INEX7,INEX8,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,455635601,V,3428,V08,SKBIOELG,08/2014,1,07/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-08,NaN,09/2014
1,455647401,V,3429,V08,SKBIOELG,08/2014,1,07/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-08,NaN,09/2014
2,455667301,V,3430,V08,SKBIOELG,09/2014,1,08/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-08,NaN,09/2014
3,455864401,V,3444,V07,SKBIOELG,08/2014,1,08/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-09,NaN,09/2014
4,456431901,V,3431,V08,SKBIOELG,09/2014,1,09/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-12,NaN,09/2014


### ISPC_Labs: useful - Feat-bio

In [83]:
IPSCLAB=dict_files[files_names[5]]
IPSCLAB.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PDMEDYN,PDMEDDT,PDMEDTM,WARFBEF,...,CPTSPNRT,CPTSPNDR,CPTRMTMP,BLDSHPDT,SHPCGLP,CDIID,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,551588601,V,3409,U01,IPSCLAB,01/2016,1,01/2016,08:25:00,0,...,1500.0,10.0,1.0,01/2016,1.0,11287.1,01/2016,2016-01-26,NaN,01/2016
1,551594301,V,4051,U01,IPSCLAB,01/2016,1,01/2016,08:31:00,0,...,1500.0,10.0,1.0,01/2016,1.0,11289.1,01/2016,2016-01-26,NaN,01/2016
2,551939601,V,4105,U01,IPSCLAB,01/2016,0,NaN,NaN,0,...,1500.0,10.0,1.0,01/2016,1.0,11291.1,01/2016,2016-01-28,NaN,01/2016
3,554801901,V,3411,U01,IPSCLAB,02/2016,0,NaN,NaN,0,...,1500.0,10.0,1.0,02/2016,1.0,11295.1,02/2016,2016-02-15,NaN,02/2016
4,555320201,V,3419,U01,IPSCLAB,02/2016,0,NaN,NaN,0,...,1500.0,10.0,1.0,02/2016,1.0,11294.1,02/2016,2016-02-16,NaN,02/2016


### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio

In [89]:
PILOTRSLT=dict_files[files_names[6]]
PILOTRSLT.head()

,PATNO,GENDER,DIAGNOSIS,CLINICAL_EVENT,TYPE,TESTNAME,TESTVALUE,UNITS,RUNDATE,PROJECTID,PI_NAME,PI_INSTITUTION,update_stamp
0,3400,Female,PD,BL,Cerebrospinal Fluid,Abeta 42,207,pg/ml,2011-07-21,101,Les Shaw,University of Pennsylvania,2011-11-08 13:45:04.0
1,3400,Female,PD,BL,Cerebrospinal Fluid,p-Tau181P,14,pg/ml,2011-07-21,101,Les Shaw,University of Pennsylvania,2011-11-08 13:45:04.0
2,3400,Female,PD,BL,Cerebrospinal Fluid,Total tau,10,pg/ml,2011-07-21,101,Les Shaw,University of Pennsylvania,2011-11-08 13:45:04.0
3,3400,Female,PD,ST,Cerebrospinal Fluid,Abeta 42,231,pg/ml,2011-07-15,101,Les Shaw,University of Pennsylvania,2011-11-08 13:45:04.0
4,3400,Female,PD,ST,Cerebrospinal Fluid,p-Tau181P,21,pg/ml,2011-07-15,101,Les Shaw,University of Pennsylvania,2011-11-08 13:45:04.0


### Laboratory_Procedures - useless

In [90]:
LAB=dict_files[files_names[7]]
LAB.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,LMDT,LMTM,FASTSTAT,PDMEDYN,...,BSVAFT,BSALQN,BSFTM,BSSTTMP,PLASBFCT,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226845701,V,3400,SC,LAB,06/2010,06/2010,07:00:00,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,06/2010,2010-07-14 10:27:53.0,NaN,07/2010
1,226877401,V,3401,SC,LAB,06/2010,06/2010,06:45:00,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,06/2010,2010-07-14 10:28:44.0,NaN,06/2010
2,226894001,V,3402,SC,LAB,06/2010,06/2010,08:15:00,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,06/2010,2010-07-14 10:29:36.0,NaN,06/2010
3,226921901,V,3403,SC,LAB,06/2010,06/2010,08:00:00,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,06/2010,2010-07-14 10:31:53.0,NaN,07/2010
4,231766401,V,3403,BL,LAB,07/2010,07/2010,12:45:00,NaN,0.0,...,4.5,3.0,16:30:00,-80.0,NaN,NaN,07/2010,2014-03-27 09:21:49.0,NaN,07/2010


### Laboratory_Procedures_with_Elapsed_Times - useless

In [92]:
LABELAPSE=dict_files[files_names[8]]
LABELAPSE.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,LMDT,LMTM,PDMEDYN,PDMEDDT,...,SITE_APRV,DurUT1TM,DurUT1SPNTM,DurUT1FFTM,DurPLASTM,DurPLASSPNTM,DurPLASFFTM,DurBLDSERTM,DurSERSPNTM,DurSERFFTM
0,226845701,V,3400,SC,LAB,06/2010,06/2010,07:00:00,0.0,NaN,...,07/2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,226877401,V,3401,SC,LAB,06/2010,06/2010,06:45:00,0.0,NaN,...,06/2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,226894001,V,3402,SC,LAB,06/2010,06/2010,08:15:00,0.0,NaN,...,06/2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,226921901,V,3403,SC,LAB,06/2010,06/2010,08:00:00,0.0,NaN,...,07/2010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,231766401,V,3403,BL,LAB,07/2010,07/2010,12:45:00,0.0,NaN,...,07/2010,75.0,18.0,45.0,200.0,5.0,25.0,200.0,5.0,25.0


### Lumbar_Puncture_Sample_Collection - useful: Feat-bio

In [93]:
LUMBAR=dict_files[files_names[9]]
LUMBAR.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,LMDT,LMTM,FASTSTAT,PDMEDYN,...,FLUORO,FLUORODT,SPFI,SPFIDT,LPNDRSN,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232556301,V,3401,BL,LUMBAR,07/2010,07/2010,12:30:00,NaN,0.0,...,0.0,NaN,0.0,NaN,NaN,NaN,07/2010,2014-03-27 08:41:14.0,NaN,07/2010
1,232568401,V,3403,BL,LUMBAR,07/2010,07/2010,12:45:00,NaN,0.0,...,0.0,NaN,0.0,NaN,NaN,NaN,07/2010,2014-03-27 09:20:12.0,NaN,07/2010
2,233649201,V,3400,BL,LUMBAR,07/2010,07/2010,14:20:00,NaN,0.0,...,0.0,NaN,0.0,NaN,NaN,subject ate a bag of pretzels during her visit...,07/2010,2017-01-05 09:15:33.0,NaN,07/2010
3,235052301,V,3404,BL,LUMBAR,07/2010,07/2010,05:45:00,NaN,0.0,...,0.0,NaN,0.0,NaN,NaN,Total CSF aliquotted after spinning = 13.5 ml,07/2010,2014-03-27 10:38:12.0,NaN,07/2010
4,236619201,V,3405,BL,LUMBAR,07/2010,07/2010,19:00:00,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,"LP attempted, fluid not taken",08/2010,2012-06-18 09:05:00.0,NaN,08/2010


### IUSM_BIOSPECIMEN_CELL_CATALOG - DNK
#### Look @ the PPMI doc to understand what is this table. 

In [98]:
IUSMBIOCELLCAT=dict_files[files_names[10]]
IUSMBIOCELLCAT.head()

,SPECIMEN_NO,ALIAS_ID,APPRDX,CLINICAL_EVENT,TYPE,NUM_AVAILABLE,QUANTITY,QUANTITY_UNITS,ADDTL_STOCK_AVAIL_ON_REQ,VENDOR,...,CELLS_PER_CONTAINER,PASSAGE_NUMBER_AT_FREEZE,KARYOTYPE,KARYOTYPE_PROCEDURE,IDENTITY_TEST,PLURIPOTENCY,MYCOPLASMA,STERILITY_TEST,FREEZING_MEDIA,NONCONFORMANCE
0,900077,12593,4,U01,Cell Line,40,1,x10^6 cells/1ml,N,CDI,...,NaN,13,37188.0,G-banding karyotype,Pass,Pass,Pass,NaN,NaN,NaN
1,860305,18567,4,U01,Cell Line,18,1,x10^6 cells/1ml,N,CDI,...,NaN,10,37189.0,G-banding karyotype,Pass,Pass,Pass,NaN,NaN,NaN
2,540028,3186,1,U01,Cell Line,38,1,x10^6 cells/1ml,N,CDI,...,NaN,10,37188.0,G-banding karyotype,Pass,Pass,Pass,NaN,NaN,NaN
3,748489,3220,1,U01,Cell Line,58,1,x10^6 cells/1ml,N,CDI,...,NaN,10,37188.0,G-banding karyotype,Pass,Pass,Pass,NaN,NaN,NaN
4,487431,3234,1,U01,Cell Line,58,1,x10^6 cells/1ml,N,CDI,...,NaN,10,37189.0,G-banding karyotype,Pass,Pass,Pass,NaN,NaN,NaN


### Deprecated_Biospecimen_Analysis_Results - useless

In [97]:
DEPRSLT=dict_files[files_names[11]]
#DEPRSLT.head()

### Blood_Chemistry__Hematology - DNK
Look @ the PPMI doc to understand what is this table.

In [102]:
COVANCE=dict_files[files_names[12]]
COVANCE.head()

,PATNO,EVENT_ID,PAG_NAME,LCOLLDT,COLLTM,LRECDT,RECTM,LRPTDT,RPTTM,LABCODE,...,LSIRES,LSIUNIT,LSILORNG,LSIHIRNG,LUSRES,LUSUNIT,LUSLORNG,LUSHIRNG,LRESFLG,LTSTCOMM
0,3404,SC,COVANCE,06/2010,10:30:00,06/2010,04:56:00,06/2010,22:14:00,COVANCE,...,10.1,sec,9.70,12.30,10.1,sec,9.70,12.30,NaN,NaN
1,3404,SC,COVANCE,06/2010,10:30:00,06/2010,04:56:00,06/2010,22:14:00,COVANCE,...,21.0,sec,22.80,31.00,21.0,sec,22.80,31.00,L,NaN
2,3404,SC,COVANCE,06/2010,10:30:00,06/2010,04:56:00,06/2010,17:28:00,COVANCE,...,0.28,GI/L,0.12,0.92,0.28,x10^3/uL,0.12,0.92,NaN,NaN
3,3404,SC,COVANCE,06/2010,10:30:00,06/2010,04:56:00,06/2010,17:28:00,COVANCE,...,0.09,GI/L,0.00,0.57,0.09,x10^3/uL,0.00,0.57,NaN,NaN
4,3404,SC,COVANCE,06/2010,10:30:00,06/2010,04:56:00,06/2010,17:28:00,COVANCE,...,0.03,GI/L,0.00,0.20,0.03,x10^3/uL,0.00,0.20,NaN,NaN


### DNA_Sample_Collection - Very likely useless

In [106]:
DNA=dict_files[files_names[13]]
DNA.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,BLDDNA,BLDDNADT,BLDVOLML,DNASHPDT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226892801,V,3402,SC,DNA,06/2010,1.0,06/2010,8.0,06/2010,06/2010,2010-06-21 10:52:54.0,NaN,06/2010
1,226841801,V,3400,SC,DNA,06/2010,1.0,06/2010,8.0,06/2010,06/2010,2010-06-21 08:52:15.0,NaN,06/2010
2,226957601,V,3404,SC,DNA,06/2010,1.0,06/2010,8.0,06/2010,06/2010,2010-06-21 12:40:08.0,NaN,07/2010
3,226921101,V,3403,SC,DNA,06/2010,1.0,06/2010,8.0,06/2010,06/2010,2010-06-21 11:26:51.0,NaN,07/2010
4,232503301,V,3405,SC,DNA,07/2010,1.0,07/2010,8.0,07/2010,07/2010,2010-07-13 12:40:18.0,NaN,07/2010


### IUSM_Catalog - DNK
Look @ the PPMI doc to understand what is this table

In [111]:
IUSMCAT=dict_files[files_names[14]]
IUSMCAT.head()

,SPECIMEN_NO,ALIAS_ID,APPRDX,CLINICAL_EVENT,TYPE,NUM_AVAILABLE,QUANTITY,QUANTITY_UNITS,ADDTL_STOCK_AVAIL_ON_REQ,MASS_UG,CONCENTRATION,RATIO_260_280,RIN,RIN_ROBOT,QC_CLASS_RNA,QC_STATUS_DNA_RNA,CLOTTING,TURBIDITY,AVERAGE_HEMOGLOBIN
0,780844.b,10874,4,V07,PBMC,3,2.56,MMcells/ml,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,423441,10874,4,V05,Plasma,8,200.00,ul,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-Clear,26.180
2,642804,10874,4,V06,DNA,1,NaN,NaN,Y,90.3,0.121,1.94,NaN,NaN,NaN,QC complete,NaN,NaN,NaN
3,191585,10874,4,V03,Plasma,6,200.00,ul,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0-Clear,19.562
4,642794,10874,4,V06,RNA,2,NaN,NaN,Y,1.0,52.500,2.10,9.0,NaN,High,QC Complete,NaN,NaN,NaN


## 3-Enrollment - useless?

In [289]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/3-Enrollment/'
# saving all the csv files in a dataframe
import os
import glob
# glob.glob('*.csv') #find all the csv files in a pathname. 
os.chdir(folder_path)
csv_files = [i for i in glob.glob('*.csv')]
csv_files

# Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
dict_files={}
files_names=[]
for files in csv_files:
#for i, files in enumerate(csv_files): 
    df=pd.read_csv(files,engine='python')
    basepath, filename=ntpath.split(files)
    files_names.append(filename)
    dict_files[filename]=df


# Calculating the # of rows and features for each dataframe and storing them in a list of tuples
shape_df=[]
for i in range(0,len(dict_files.keys())):
    nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
    shape_df.append((nrows,ncols))
print(shape_df)
print(list(dict_files.keys()))

# DF TO EXCEL
#from pandas import ExcelWriter

#writer = ExcelWriter('PythonExport.xlsx')
#yourdf.to_excel(writer,'Sheet5')
#writer.save()

[(110, 16), (1310, 13), (345, 14), (203, 3), (26, 20), (79, 12), (2067, 37), (42, 19), (3734, 12), (34, 12), (21257, 18), (1084, 6), (2630, 13), (4975, 12), (33, 20)]
['Florbetaben_Eligibility.csv', 'Contact_Information_Brain_Bank.csv', 'Conclusion_of_Study_Participation.csv', 'ST_CATALOG.csv', 'Skin_Biopsy_Eligibility.csv', 'RBD_PSG_Eligibility.csv', 'Inclusion_Exclusion.csv', 'AV-133_Subject_Eligibility.csv', 'Primary_Diagnosis.csv', 'TAP-PD_Conclusion.csv', 'Signature_Form.csv', 'FOUND_Enrollment_Status.csv', 'Contact_Information_FOUND.csv', 'Research_Advance_Directive.csv', 'TAP-PD_Subject_Eligibility.csv']


### 'Florbetaben_Eligibility.csv' - useless

In [291]:
FBELIG=dict_files[files_names[0]]
FBELIG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,SIGNCNST,FBCNSTDT,INEX1,INEX2,INEX3,INEX4,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,455171501,V,90456,V04,FBELIG,08/2014,1,08/2014,1,0,0,0,09/2014,2014-09-03,NaN,09/2014
1,455552701,V,3428,V08,FBELIG,07/2014,1,07/2014,1,0,0,0,09/2014,2014-09-05,NaN,09/2014
2,455651401,V,3429,V08,FBELIG,08/2014,1,07/2014,1,0,0,0,09/2014,2014-09-08,NaN,09/2014
3,456366301,V,3431,V08,FBELIG,09/2014,1,09/2014,1,0,0,0,09/2014,2014-09-12,NaN,09/2014
4,458661901,V,3430,V08,FBELIG,09/2014,1,08/2014,1,0,0,0,09/2014,2014-09-22,NaN,09/2014


### 'Contact_Information_Brain_Bank.csv' - useless

In [293]:
BBNKINF=dict_files[files_names[1]]
BBNKINF.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,BBNKSHR,BBNKINDT,BBNKXFDT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,579722501,V,3403,V13,BBNKINF,07/2016,0.0,NaN,NaN,07/2016,2016-07-21,NaN,07/2016
1,579849801,V,4108,V09,BBNKINF,07/2016,1.0,07/2016,07/2016,07/2016,2016-07-22,NaN,07/2016
2,580293501,V,55349,CONSENT,BBNKINF,07/2016,0.0,NaN,NaN,07/2016,2016-07-26,NaN,07/2016
3,580611701,V,3215,V12,BBNKINF,07/2016,1.0,07/2016,07/2016,07/2016,2016-07-28,NaN,07/2016
4,581597601,V,4058,V10,BBNKINF,07/2016,0.0,NaN,NaN,08/2016,2016-08-02,NaN,08/2016


### 'Conclusion_of_Study_Participation.csv' - very likely useless
#### Only give the final comment on the patient for his final visit

In [296]:
CONCL=dict_files[files_names[2]]
CONCL.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,COMPLT,WDRSN,WDRSNCM,WDDT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,238261101,V,3405,FNL,CONCL,08/2010,0,15.0,"LP attempted, fluid not taken. Subject unwill...",08/2010,08/2010,2010-08-11 08:18:24.0,NaN,08/2010
1,295436201,V,3007,FNL,CONCL,06/2011,0,1.0,NaN,06/2011,06/2011,2011-06-06 09:41:23.0,NaN,06/2011
2,357901601,V,4009,FNL,CONCL,08/2012,0,2.0,NaN,08/2012,08/2012,2012-08-10 13:46:07.0,NaN,08/2012
3,339098901,V,3809,FNL,CONCL,03/2012,0,3.0,Subject stopped her participation due to priva...,03/2012,03/2012,2012-03-21 08:18:24.0,NaN,03/2012
4,311484501,V,3858,FNL,CONCL,09/2011,0,15.0,patient decided to take anti-parkinsonian medi...,09/2011,09/2011,2011-09-09 03:24:25.0,NaN,09/2011


### 'ST_CATALOG.csv' - very likely useless

In [300]:
STCATALOG=dict_files[files_names[3]]
STCATALOG.head()

,PATNO,STRPLCVS,STVIS
0,3400,V02,1
1,3403,V02,1
2,3406,V02,1
3,3502,V02,1
4,3056,V02,1


### 'Skin_Biopsy_Eligibility.csv' - USELESS

In [302]:
SKBIOELG=dict_files[files_names[4]]
SKBIOELG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,SIGNCNST,SKBICNDT,INEX1,INEX2,INEX3,INEX4,INEX5,INEX6,INEX7,INEX8,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,455635601,V,3428,V08,SKBIOELG,08/2014,1,07/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-08,NaN,09/2014
1,455647401,V,3429,V08,SKBIOELG,08/2014,1,07/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-08,NaN,09/2014
2,455667301,V,3430,V08,SKBIOELG,09/2014,1,08/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-08,NaN,09/2014
3,455864401,V,3444,V07,SKBIOELG,08/2014,1,08/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-09,NaN,09/2014
4,456431901,V,3431,V08,SKBIOELG,09/2014,1,09/2014,1,1,1,1,0,0,0,0,09/2014,2014-09-12,NaN,09/2014


### 'RBD_PSG_Eligibility.csv' - useful as it measures sleep disorders

In [304]:
ELGBITY=dict_files[files_names[5]]
ELGBITY.head()

,PATNO,protocol_id,psg_eval_date,rbd_eval_date,rbd_eligibility_results,date_eligibility_report,percent_rem_sleep,rem_sleep_without_atonia,rem_sleep_scored,percent_withoutAtonia_epochs,percent_withoutAtonia_miniEpochs,psg_eval_activity
0,60002,P-PPMI Cohort,06/2010,07/2013,Eligible,07/2013,NaN,3,2,NaN,NaN,1
1,60003,P-PPMI Cohort,03/2007,03/2014,Not Eligible,03/2014,20.0,3,3,NaN,NaN,2
2,60004,P-PPMI Cohort,02/2014,05/2014,Eligible,05/2014,16.0,3,1,NaN,NaN,1
3,60005,P-PPMI Cohort,04/2001,03/2014,Eligible,03/2014,15.0,3,1,NaN,NaN,1
4,60006,P-PPMI Cohort,12/2010,03/2014,Eligible,03/2014,23.0,3,2,NaN,NaN,1


### 'Inclusion_Exclusion.csv' - useless

In [307]:
INEXPD=dict_files[files_names[6]]
INEXPD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,INEX1,INEX2,INEX3,INEX4,...,INEX31,INEX32,INEX33,INEX34,INEX35,WAIVERCD,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232520801,V,3401,SC,INEXHC,07/2010,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-13 13:14:33.0,NaN,07/2010
1,232591801,V,3400,SC,INEXPD,07/2010,1.0,1.0,1.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,07/2010,2010-07-14 10:11:22.0,NaN,07/2010
2,232587601,V,3405,SC,INEXHC,07/2010,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-09-14 08:09:18.0,NaN,07/2010
3,233879001,V,3404,SC,INEXHC,07/2010,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-20 09:08:19.0,NaN,07/2010
4,235907901,V,3406,SC,INEXPD,07/2010,1.0,1.0,1.0,1.0,...,0.0,0.0,NaN,NaN,NaN,NaN,07/2010,2010-07-29 13:18:51.0,NaN,07/2010


### 'AV-133_Subject_Eligibility.csv' - useless

In [310]:
AVELIG=dict_files[files_names[7]]
AVELIG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,SIGNCNST,AVCNSTDT,INEX1,INEX2,INEX3,INEX4,INEX5,INEX6,INEX7,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,367319801,V,3662,SC,AVELIG,08/2012,1,08/2012,NaN,0,0,0,0,0,0,10/2012,2012-10-16,NaN,10/2012
1,374250201,V,4102,SC,AVELIG,11/2012,1,11/2012,NaN,0,0,0,0,0,0,11/2012,2012-11-27,NaN,11/2012
2,361667101,V,4091,SC,AVELIG,08/2012,1,08/2012,NaN,0,0,0,0,0,0,09/2012,2012-09-10,NaN,09/2012
3,361662601,V,4096,SC,AVELIG,08/2012,1,08/2012,1.0,0,0,0,0,0,0,09/2012,2015-10-19,NaN,09/2012
4,372010601,V,4100,SC,AVELIG,11/2012,1,11/2012,NaN,0,0,0,0,0,0,11/2012,2012-11-14,NaN,11/2012


### 'Primary_Diagnosis.csv' - useful: Feat-Gen

In [314]:
PRIMDXPD=dict_files[files_names[8]]
PRIMDXPD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PRIMDIAG,OTHNEURO,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226859501,V,3401,SC,PRIMDXPD,06/2010,17.0,NaN,06/2010,2010-06-21 09:50:56.0,NaN,06/2010
1,232502501,V,3405,SC,PRIMDXPD,07/2010,17.0,NaN,07/2010,2010-07-13 12:36:21.0,NaN,07/2010
2,226902401,V,3403,SC,PRIMDXPD,06/2010,1.0,NaN,06/2010,2010-06-21 11:04:02.0,NaN,07/2010
3,232739901,V,3404,SC,PRIMDXPD,06/2010,17.0,NaN,07/2010,2010-07-14 09:41:24.0,NaN,07/2010
4,232697801,V,3406,SC,PRIMDXPD,07/2010,1.0,NaN,07/2010,2010-07-14 07:46:16.0,NaN,07/2010


### 'TAP-PD_Conclusion.csv' - very likely useless

In [316]:
TAPCONCL=dict_files[files_names[9]]
TAPCONCL.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,COMPLT,WDDT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,317940801,V,3432,BL,TAPCONCL,10/2011,0,10/2011,10/2011,2011-10-25 06:24:17.0,NaN,10/2011
1,369469801,V,3435,FNL,TAPCONCL,10/2012,1,NaN,10/2012,2012-10-26 12:45:28.0,NaN,10/2012
2,363731701,V,3432,FNL,TAPCONCL,08/2012,1,NaN,09/2012,2012-09-24 12:51:05.0,NaN,09/2012
3,364245701,V,3433,FNL,TAPCONCL,09/2012,1,NaN,09/2012,2012-09-27 08:32:45.0,NaN,09/2012
4,378945401,V,3220,V04,TAPCONCL,12/2012,1,NaN,12/2012,2012-12-19 13:13:53.0,NaN,12/2012


### 'Signature_Form.csv' - most likely useful: Meds
#### Can be useful for the meds log

In [319]:
SIG=dict_files[files_names[10]]
SIG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,VISSTAT,VISSTCM,VISMSRSN_CODE,ASSESCMP,LGCMED2,LGAE2,LGCONDTN,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233160901,V,3400,SC,SIG,07/2010,1,NaN,NaN,1.0,1.0,3.0,3.0,NaN,07/2010,2016-07-05 08:34:06.0,NaN,07/2010
1,232523801,V,3401,SC,SIG,07/2010,1,NaN,NaN,1.0,1.0,3.0,3.0,NaN,07/2010,2010-07-13 13:21:48.0,NaN,07/2010
2,233845301,V,3401,BL,SIG,07/2010,4,NaN,NaN,1.0,1.0,1.0,3.0,MR completed outside of window due to subject'...,07/2010,2010-09-21 08:29:06.0,NaN,07/2010
3,234863801,V,3400,BL,SIG,07/2010,4,NaN,NaN,1.0,2.0,3.0,3.0,Visit out of window due to subject's schedule,07/2010,2010-09-14 06:09:53.0,NaN,07/2010
4,236050301,V,3404,BL,SIG,07/2010,4,NaN,NaN,1.0,2.0,3.0,1.0,BL visit occurred out of window due to a scree...,07/2010,2010-09-21 08:41:48.0,NaN,07/2010


### 'FOUND_Enrollment_Status.csv' - useless

In [322]:
FOUND=dict_files[files_names[11]]
FOUND.head()

,PATNO,FOSTATUS,FOCONSENTDT,FOFOLLOWST,FOFOLLOWMO,LAST_UPDATE
0,12224,1,09/2015,14,31.0,2018-04-29
1,12499,1,08/2016,13,20.0,2018-05-04
2,12593,1,12/2015,13,NaN,2018-04-06
3,13039,1,10/2015,14,29.0,2018-03-09
4,13424,1,10/2017,1,NaN,2017-10-26


### 'Contact_Information_FOUND.csv' - useless

In [325]:
FOUNDINF=dict_files[files_names[12]]
FOUNDINF.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,FNDSHR,FNDINFDT,FNDXFRDT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,451971601,V,3428,V08,FOUNDINF,07/2014,1.0,07/2014,08/2015,08/2014,2015-08-05,NaN,08/2014
1,451979601,V,3429,V08,FOUNDINF,07/2014,1.0,07/2014,08/2015,08/2014,2015-08-05,NaN,08/2014
2,451989101,V,4053,V06,FOUNDINF,07/2014,1.0,07/2014,12/2015,08/2014,2015-12-21,NaN,08/2014
3,452617201,V,3444,V07,FOUNDINF,08/2014,1.0,08/2014,08/2015,08/2014,2015-08-05,NaN,08/2014
4,456378101,V,3431,V08,FOUNDINF,09/2014,1.0,09/2014,08/2015,09/2014,2015-08-05,NaN,09/2014


### 'Research_Advance_Directive.csv' - useless

In [327]:
RAD=dict_files[files_names[13]]
RAD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,RADSTAT,RADSTADT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,451382401,V,3430,V08,RAD,08/2014,1.0,08/2014,08/2014,2014-08-07,NaN,08/2014
1,451411201,V,3403,V10,RAD,08/2014,3.0,NaN,08/2014,2014-08-07,NaN,08/2014
2,451737401,V,4093,V06,RAD,08/2014,3.0,NaN,08/2014,2014-08-11,NaN,08/2014
3,451972201,V,3428,V08,RAD,07/2014,1.0,07/2014,08/2014,2014-08-12,NaN,08/2014
4,451980301,V,3429,V08,RAD,07/2014,1.0,07/2014,08/2014,2014-08-12,NaN,08/2014


### 'TAP-PD_Subject_Eligibility.csv' - useless

In [329]:
TAPELIG=dict_files[files_names[14]]
TAPELIG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,SIGNCNST,TAPCNSDT,INEX1,INEX2,INEX3,INEX4,INEX5,INEX6,TAPENRDT,DEVICESN,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,312686201,V,3432,CONSENT,TAPELIG,09/2011,1,09/2011,1,1,1,1,0,0,09/2011,HVDG9341016,09/2011,2011-11-18 13:01:58.0,NaN,09/2011
1,314756401,V,3433,CONSENT,TAPELIG,09/2011,1,09/2011,1,1,1,1,0,0,09/2011,HVDG9341006,09/2011,2011-11-18 12:59:59.0,NaN,09/2011
2,318524101,V,3435,BL,TAPELIG,10/2011,1,10/2011,1,1,1,1,0,0,10/2011,HVDG9353016,10/2011,2011-11-18 13:03:41.0,NaN,10/2011
3,319998101,V,3218,CONSENT,TAPELIG,10/2011,1,10/2011,1,1,1,1,0,0,10/2011,HVDG9341013,11/2011,2011-12-09 11:59:42.0,NaN,11/2011
4,322801001,V,3436,BL,TAPELIG,11/2011,1,11/2011,1,1,1,1,0,0,11/2011,HVDG9343009,12/2011,2011-12-01 10:01:23.0,NaN,12/2011


## 4-Imaging - dataset not to be used for the time being.

In [112]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/4-Imaging/'
# saving all the csv files in a dataframe
import os
import glob
# glob.glob('*.csv') #find all the csv files in a pathname. 
os.chdir(folder_path)
csv_files = [i for i in glob.glob('*.csv')]
csv_files

# Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
dict_files={}
files_names=[]
for files in csv_files:
#for i, files in enumerate(csv_files): 
    df=pd.read_csv(files,engine='python')
    basepath, filename=ntpath.split(files)
    files_names.append(filename)
    dict_files[filename]=df


# Calculating the # of rows and features for each dataframe and storing them in a list of tuples
shape_df=[]
for i in range(0,len(dict_files.keys())):
    nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
    shape_df.append((nrows,ncols))
print(shape_df)
print(list(dict_files.keys()))

# DF TO EXCEL
#from pandas import ExcelWriter

#writer = ExcelWriter('PythonExport.xlsx')
#yourdf.to_excel(writer,'Sheet5')
#writer.save()

[(88, 37), (81, 34), (87, 32), (2483, 18), (2115, 12), (80, 11), (2125, 22), (1200, 14), (933, 3), (102, 25), (1342, 7)]
['FBB_Analysis_Data.csv', 'AV-133_Image_Metadata.csv', 'AV-133_Imaging.csv', 'DaTscan_Imaging.csv', 'SPECT_Scan_Information_Source_Document.csv', 'AV-133_SBR_Results.csv', 'Magnetic_Resonance_Imaging.csv', 'DTI_Regions_of_Interest.csv', 'DaTSCAN_SPECT_Visual_Interpretation_Assessment.csv', 'FBB_Metadata.csv', 'MRI_Imaging_Data_Transfer_Information_Source_Document.csv']


### FBB_Analysis_Data.csv

In [116]:
FBBANAL=dict_files[files_names[0]]
FBBANAL.head()

,PATNO,EVENT_ID,SCAN_DATE,cerebellar_cortex_r,cerebellar_cortex_l,mesial_temporal_cortex_r,mesial_temporal_cortex_l,pons,subcortical_white_matter,cerebellar_white_matter,...,lateral_temporal_cortex_l,orbitofrontal_cortex_r,orbitofrontal_cortex_l,frontal_cortex_r,frontal_cortex_l,temporal_cortex_r,temporal_cortex_l,composite_summary,mean_cerebellum_gray,mean_whole_cerebellum
0,3433,V04,10/2014,1.03,0.97,1.27,1.21,1.89,2.26,2.07,...,1.27,1.30,1.30,1.34,1.37,1.31,1.26,1.32,1.0,1.08
1,4100,V06,12/2014,0.99,1.01,1.45,1.38,2.25,2.45,2.49,...,1.42,1.48,1.41,1.49,1.49,1.43,1.41,1.40,1.0,1.11
2,4105,V06,02/2015,1.01,0.99,1.36,1.29,1.98,2.19,2.37,...,1.29,1.33,1.33,1.34,1.37,1.33,1.29,1.39,1.0,1.08
3,4108,V06,02/2015,1.05,0.95,1.24,1.25,1.88,2.32,2.25,...,1.27,1.38,1.32,1.38,1.36,1.33,1.27,1.36,1.0,1.09
4,4116,V06,06/2015,0.99,1.01,1.22,1.15,1.84,1.96,2.21,...,1.24,1.19,1.24,1.21,1.25,1.22,1.22,1.23,1.0,1.10


### AV-133_Image_Metadata.csv

In [118]:
AV133META=dict_files[files_names[1]]
AV133META.head()

,PATNO,protocol_id,scan_date,EVENT_ID,weight,weight_unit,ligand,batch_number,injection_site,pass_qc,...,trans_two_file_name,trans_two_start_time_hh,trans_two_start_time_mm,emiss_two_file_name,emiss_two_start_time_hh,emiss_two_start_time_mm,scan_quality_rating_pet,pet_image_acceptable,comments,SCAN_DATE_SERIAL
0,4102,PPMI,01/2015,V06,96.81,kg,18F-AV-133,01V012215-1,RT AC,Yes,...,NaN,15,39,NaN,15,46,1,Yes,NaN,1696
1,4091,PPMI,10/2016,V10,59.09,kg,18F-AV-133,01V100316-1,RT AC,Yes,...,NaN,15,35,NaN,15,46,1,Yes,NaN,2316
2,50192,PPMI,05/2016,V06,73.90,kg,18F-AV-133,AV13320160531-1,RT ACV,Yes,...,NaN,16,8,NaN,16,11,1,Yes,NaN,2191
3,3662,PPMI,10/2012,SC,75.00,kg,18F-AV-133,AV13320121009-1,RTAW,Yes,...,154-3662,15,2,154-3662,15,3,1,Yes,NaN,861
4,3234,PPMI,06/2014,ST,79.60,kg,18F-AV-133,01V061614-1,RIGHT HAND,Yes,...,NaN,14,48,NaN,14,49,1,Yes,NaN,1476


### AV-133_Imaging.csv

In [120]:
AVIMAG=dict_files[files_names[2]]
AVIMAG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,INVEVLBF,VITLTMBF,SYSSUPBF,DIASUPBF,...,PETCMPLT,PETDT,INVEVLAF,XFRYN,VMATRSLT,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,361667301,V,4091,SC,AVIMAG,08/2012,1,15:48:00,113.0,67.0,...,1.0,08/2012,1.0,0.0,1.0,NaN,09/2012,2012-11-06,NaN,10/2012
1,361664501,V,4096,SC,AVIMAG,08/2012,1,16:11:00,119.0,81.0,...,1.0,08/2012,1.0,1.0,1.0,subject denies childbearing potential due to n...,09/2012,2012-10-25,NaN,09/2012
2,368445301,V,3662,SC,AVIMAG,10/2012,1,13:40:00,142.0,88.0,...,1.0,10/2012,1.0,1.0,2.0,NaN,10/2012,2012-10-22,NaN,10/2012
3,369102001,V,4098,SC,AVIMAG,10/2012,1,16:38:00,143.0,86.0,...,1.0,10/2012,1.0,1.0,1.0,NaN,10/2012,2012-10-31,NaN,10/2012
4,377945201,V,4102,SC,AVIMAG,12/2012,1,17:15:00,125.0,72.0,...,1.0,NaN,NaN,1.0,1.0,NaN,12/2012,2012-12-20,NaN,12/2012


### DaTscan_Imaging.csv

In [122]:
DATSCAN=dict_files[files_names[3]]
DATSCAN.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,DATSCAN,DATSCNDT,SCNLOC,SCNINJCT,DATXFRYN,VSINTRPT,COMM,VSRPTELG,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232521601,V,3401,SC,DATSCAN,07/2010,1,07/2010,1.0,NaN,1.0,2,NaN,NaN,07/2010,2012-06-18 12:02:21.0,NaN,07/2010
1,234574801,V,3404,SC,DATSCAN,07/2010,1,07/2010,1.0,NaN,1.0,2,NaN,NaN,07/2010,2012-06-18 12:04:52.0,NaN,07/2010
2,231793201,V,3402,SC,DATSCAN,07/2010,1,07/2010,1.0,NaN,1.0,2,NaN,NaN,07/2010,2012-06-18 12:03:52.0,NaN,07/2010
3,232591601,V,3400,SC,DATSCAN,07/2010,1,07/2010,1.0,NaN,1.0,1,NaN,NaN,07/2010,2012-06-18 09:57:01.0,NaN,07/2010
4,235907501,V,3406,SC,DATSCAN,07/2010,1,07/2010,1.0,NaN,1.0,1,NaN,NaN,07/2010,2012-06-18 12:06:05.0,NaN,07/2010


### SPECT_Scan_Information_Source_Document.csv

In [129]:
SPECTSCANINFO=dict_files[files_names[4]]
SPECTSCANINFO.head()

,study_subject_id,03_visit_number_E1_C2,04_scan_acquisition_date_spect_E1_C2,05_scan_start_time_E1_C2,06_date_injected_E1_C2,07_time_injected_E1_C2,08_actual_dose_injected_E1_C2,08a_actual_dose_unit_E1_C2,09_scan_quality_rating_spect_E1_C2,10_spect_image_acceptable_E1_C2,comment_E1_C7,other_E1_C7
0,3400,SC,07/2010,14:26:00,07/2010,10:26:00,5.16,mCi,1.0,Yes,NaN,NaN
1,3400,V04,07/2011,15:10:00,07/2011,11:15:00,4.90,mCi,1.0,Yes,NaN,NaN
2,3400,V06,07/2012,15:12:00,07/2012,11:16:00,4.98,mCi,1.0,Yes,NaN,NaN
3,3400,V10,07/2014,15:46:00,07/2014,12:01:00,4.89,mCi,2.0,Yes,NaN,NaN
4,3401,SC,07/2010,16:36:00,07/2010,12:36:00,5.30,mCi,1.0,Yes,NaN,NaN


### AV-133_SBR_Results.csv

In [130]:
AV133SBRRSLT=dict_files[files_names[5]]
AV133SBRRSLT.head()

,SUBJECT_NUMBER,VISIT_NUMBER,SCAN_DATE,TIMEPOINT,RCAUD-S,RPUTANT-S,RPUTPOST-S,LCAUD-S,LPUTANT-S,LPUTPOST-S,update_stamp
0,3225,SC,06/2013,Delay,1.38,1.24,0.59,1.49,1.55,0.96,2016-06-01 09:58:02.0
1,3234,ST,06/2014,Delay,1.30,0.93,0.42,1.36,1.44,0.81,2017-03-16 11:49:37.0
2,3125,V04,07/2013,Delay,1.70,1.33,0.37,1.90,2.17,1.20,2016-06-01 09:58:02.0
3,4055,V04,07/2013,Delay,0.99,0.70,0.39,1.09,1.38,0.87,2016-06-01 09:58:02.0
4,4096,SC,08/2012,Delay,0.98,0.95,0.47,1.64,1.71,1.04,2016-06-01 09:58:02.0


### Magnetic_Resonance_Imaging.csv

In [131]:
MRI=dict_files[files_names[6]]
MRI.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,MRICMPLT,MRIDT,MRIWDTI,MRIWRSS,...,PDMEDYN,ONLDOPA,ONDOPAG,ONOTHER,PDMEDDT,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,234501601,V,3400,BL,MRI,07/2010,1.0,07/2010,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,Partially empty sella and question of flatteni...,07/2010,2010-07-22 11:19:05.0,NaN,07/2010
1,235626601,V,3403,BL,MRI,07/2010,1.0,07/2010,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,07/2010,2010-07-28 11:43:54.0,NaN,07/2010
2,235629101,V,3401,BL,MRI,07/2010,1.0,07/2010,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,07/2010,2010-07-28 11:53:19.0,NaN,07/2010
3,236050101,V,3404,BL,MRI,07/2010,1.0,07/2010,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,07/2010,2010-07-30 07:21:38.0,NaN,07/2010
4,237393201,V,3405,BL,MRI,07/2010,1.0,07/2010,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,08/2010,2010-08-09 12:00:47.0,NaN,08/2010


### DTI_Regions_of_Interest

In [138]:
DTIROI=dict_files[files_names[7]]
DTIROI.head()

,PATNO,PAG_NAME,INFODT,Measure,Tissue,ROI1,ROI2,ROI3,ROI4,ROI5,ROI6,REF1,REF2,RUNDATE
0,3101,DTIROI,08/2012,E1,SN,0.000926,0.000877,0.000812,0.001246,0.001144,0.001088,0.001470,0.001434,2016-07-29
1,3101,DTIROI,08/2012,E2,SN,0.000604,0.000511,0.000587,0.001120,0.000945,0.000743,0.000537,0.000411,2016-07-29
2,3101,DTIROI,08/2012,E3,SN,0.000387,0.000337,0.000322,0.000900,0.000655,0.000463,0.000258,0.000243,2016-07-29
3,3102,DTIROI,11/2010,E1,SN,0.000926,0.000933,0.000967,0.000994,0.000971,0.001101,0.001348,0.001318,2016-07-29
4,3102,DTIROI,11/2010,E2,SN,0.000644,0.000591,0.000637,0.000531,0.000583,0.000721,0.000393,0.000269,2016-07-29


### DaTSCAN_SPECT_Visual_Interpretation_Assessment.csv

In [135]:
FBBMETA=dict_files[files_names[8]]
FBBMETA.head()

,PATNO,VISINTRP,SCANDATE
0,3400,positive,07/2010
1,3401,negative,07/2010
2,3402,negative,07/2010
3,3403,positive,07/2010
4,3404,negative,07/2010


### 'FBB_Metadata.csv'

In [141]:
FBBMETA=dict_files[files_names[9]]
FBBMETA.head()

,PATNO,EVENT_ID,SCAN_DATE,weight_derived,ligand,initial_activity_converted,initial_activity_time_hh,initial_activity_time_mm,residual_activity_converted,residual_activity_time_hh,...,trans_one_start_time_hh,trans_one_start_time_mm,trans_one_stop_time_hh,trans_one_stop_time_mm,emiss_one_start_time_hh,emiss_one_start_time_mm,emiss_one_stop_time_hh,emiss_one_stop_time_mm,scan_quality_rating_pet,pet_image_acceptable
0,3429,V08,08/2014,85.806,FBB,330.78,13,24,26.048,13,...,14,42,14,46,14,52,15,12,1,Yes
1,3430,V08,09/2014,105.328,FBB,257.52,16,6,3.034,16,...,17,31,17,35,17,40,18,0,2,Yes
2,3431,V08,09/2014,59.540,FBB,357.79,14,31,15.170,14,...,15,50,15,54,16,4,16,24,1,Yes
3,3433,V04,10/2014,62.730,FBB,351.87,12,38,23.495,12,...,13,58,14,2,14,13,14,33,1,Yes
4,3435,V08,11/2014,92.270,FBB,275.65,11,19,3.626,11,...,12,35,12,39,12,50,13,10,1,Yes


### 'MRI_Imaging_Data_Transfer_Information_Source_Document.csv'

In [144]:
MRIINFO=dict_files[files_names[10]]
MRIINFO.head()

,study_subject_id,11_scan_acquisition_date_mr_E1_C3,12_scan_quality_rating_mr_E1_C3,13_dti_sequences_E1_C3,11a_rs_mri_E1_C4,comment_E1_C7,other_E1_C7
0,3400,07/2010,2.0,No,NaN,NaN,NaN
1,3401,07/2010,2.0,No,NaN,NaN,NaN
2,3402,10/2011,2.0,No,NaN,NaN,NaN
3,3403,07/2010,2.0,No,NaN,NaN,NaN
4,3404,07/2010,1.0,No,NaN,NaN,NaN


### 5-Medical History

In [145]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
# saving all the csv files in a dataframe
import os
import glob
# glob.glob('*.csv') #find all the csv files in a pathname. 
os.chdir(folder_path)
csv_files = [i for i in glob.glob('*.csv')]
csv_files

# Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
dict_files={}
files_names=[]
for files in csv_files:
#for i, files in enumerate(csv_files): 
    df=pd.read_csv(files,engine='python')
    basepath, filename=ntpath.split(files)
    files_names.append(filename)
    dict_files[filename]=df


# Calculating the # of rows and features for each dataframe and storing them in a list of tuples
shape_df=[]
for i in range(0,len(dict_files.keys())):
    nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
    shape_df.append((nrows,ncols))
print(shape_df)
print(list(dict_files.keys()))

# DF TO EXCEL
#from pandas import ExcelWriter

#writer = ExcelWriter('PythonExport.xlsx')
#yourdf.to_excel(writer,'Sheet5')
#writer.save()

[(2407, 12), (5739, 64), (395, 35), (63933, 23), (6668, 18), (24382, 14), (22477, 29), (6217, 46), (11489, 24), (1243, 35), (428, 4), (359, 16), (3403, 23), (6218, 28), (1017, 21), (12762, 21)]
['Prodromal_Diagnostic_Questionnaire.csv', 'Diagnostic_Features.csv', 'Clinical_Diagnosis_and_Management.csv', 'General_Medical_History.csv', 'Use_of_PD_Medication.csv', 'General_Physical_Exam.csv', 'Concomitant_Medications.csv', 'General_Neurological_Exam.csv', 'Current_Medical_Conditions_Log.csv', 'Adverse_Event_Log.csv', 'Initiation_of_PD_Medication-_incidents.csv', 'Pregnancy_Form.csv', 'Surgery_for_Parkinson_Disease.csv', 'Neurological_Exam_-_Cranial_Nerves.csv', 'PD_Features.csv', 'Vital_Signs.csv']


### 'Prodromal_Diagnostic_Questionnaire.csv': useful: Feat-Gen (PRIMDIAG & PSLVL2)
#### NB: PSLV2 evaluates the proba of the patient being PD: might be useful for early PD. 

In [153]:
PRODIAG=dict_files[files_names[0]]
PRODIAG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,PRIMDIAG,OTHNEURO,PSLVL2,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,408770801,V,10362,SC,PRODDIAG,17,NaN,4,08/2013,2013-08-23 11:37:07.0,NaN,08/2013
1,605736601,V,10405,SC,PRODDIAG,17,NaN,5,12/2016,2016-12-06 12:45:38.0,NaN,12/2016
2,410987801,V,10496,SC,PRODDIAG,23,NaN,5,09/2013,2013-09-05 11:40:13.0,NaN,09/2013
3,409586301,S,10606,SC,PRODDIAG,17,NaN,4,09/2013,2015-06-26 08:19:21.0,NaN,03/2015
4,448326801,V,10662,SC,PRODDIAG,17,NaN,5,07/2014,2014-07-16 08:08:30.0,NaN,07/2014


### 'Diagnostic_Features.csv': useful: Feat-PD

In [156]:
DIAGFEAT=dict_files[files_names[1]]
DIAGFEAT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,DFSTROKE,DFRSKFCT,DFPRESNT,DFRPROG,...,DFDOPRSP,DFRAPSPE,DFBULBAR,DFCTSCAN,DFMRI,DFATYP,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,358812401,V,3428,V04,DIAGFEAT,08/2012,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0.0,08/2012,2012-08-17 12:38:06.0,NaN,08/2012
1,364251801,V,3433,V04,DIAGFEAT,09/2012,0.0,0.0,0.0,0.0,...,N,0.0,0.0,0,0,0.0,09/2012,2012-09-27 08:57:42.0,NaN,09/2012
2,366180201,V,3166,V04,DIAGFEAT,10/2012,0.0,0.0,0.0,0.0,...,N,0.0,0.0,N,0,0.0,10/2012,2012-10-09 16:35:56.0,NaN,10/2012
3,369126801,V,3162,V04,DIAGFEAT,10/2012,0.0,0.0,0.0,0.0,...,N,0.0,0.0,N,N,0.0,10/2012,2012-10-24 15:30:40.0,NaN,10/2012
4,362799201,V,3432,ST,DIAGFEAT,08/2012,0.0,0.0,0.0,0.0,...,N,0.0,0.0,0,0,0.0,09/2012,2012-09-18 13:47:13.0,NaN,09/2012


### 'Clinical_Diagnosis_and_Management.csv': Feat-PD

In [158]:
CLINDX=dict_files[files_names[2]]
CLINDX.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PSLVL,DCNOMTR,DCRTREM,DCRIGID,...,MNGTRIAL,MNGTRTOT,MNGDXTST,MNGNOTRT,OTNEURVS,NEURWKDX,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,317677601,V,3408,BL,CLINDX,10/2011,3,0,0.0,1.0,...,0.0,0.0,0.0,1.0,0,NaN,10/2011,2011-10-21 11:00:22.0,NaN,10/2011
1,317614301,V,3520,BL,CLINDX,10/2011,2,0,1.0,1.0,...,0.0,0.0,0.0,1.0,0,NaN,10/2011,2011-10-21 07:13:18.0,NaN,10/2011
2,318556601,V,3426,BL,CLINDX,10/2011,3,0,1.0,0.0,...,0.0,0.0,0.0,1.0,1,Sciatica,10/2011,2011-11-07 06:00:08.0,NaN,10/2011
3,318628801,V,3512,V02,CLINDX,10/2011,2,0,1.0,1.0,...,0.0,0.0,0.0,0.0,0,NaN,10/2011,2011-10-28 15:18:58.0,NaN,10/2011
4,318729401,V,3425,BL,CLINDX,10/2011,2,0,0.0,1.0,...,0.0,0.0,1.0,0.0,1,spinal stenosis,10/2011,2011-10-31 09:53:38.0,NaN,10/2011


### 'General_Medical_History.csv': useful: Feat-Gen

In [175]:
MHXGEN=dict_files[files_names[3]]
MHXGEN.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,MHCAT,MHROW,MHHX,MHTERM,...,LLT_NAME,PT_CODE,PT_NAME,HLT_NAME,HLGTNAME,SOCABBR1,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226698901,V,3400,SC,MHXGEN,06/2010,1f. ENT,1,1.0,Nose bleeds,...,Nose bleeds,10015090.0,Epistaxis,Nasal disorders NEC,Upper respiratory tract disorders (excl infect...,Resp,06/2010,2010-06-29 12:33:43.0,NaN,07/2010
1,226700701,V,3400,SC,MHXGEN,06/2010,1l. Gynecologic/Urologic,1,1.0,Painful menses,...,Menses painful,10013935.0,Dysmenorrhoea,Menstruation and uterine bleeding NEC,Menstrual cycle and uterine bleeding disorders,Repro,06/2010,2010-06-29 12:33:44.0,NaN,07/2010
2,226701901,V,3400,SC,MHXGEN,06/2010,1p. Neurologic (other than disease under study),1,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,06/2010,2010-06-20 05:14:14.0,NaN,07/2010
3,226702601,V,3400,SC,MHXGEN,06/2010,1r. Allergy / Immunologic (Note drug allergies),2,1.0,Indoor environmental allergens,...,Environmental allergy,10020751.0,Hypersensitivity,Allergic conditions NEC,Allergic conditions,Immun,06/2010,2010-06-29 12:33:45.0,NaN,07/2010
4,226870201,V,3401,SC,MHXGEN,06/2010,1o. Hemato/lymphatic,1,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,06/2010,2010-06-30 13:18:55.0,NaN,06/2010


### Use of PD Medication: useful: Meds

In [176]:
PDMEDUSE=dict_files[files_names[4]]
PDMEDUSE.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PDMEDYN,ONLDOPA,ONDOPAG,ONOTHER,FULNUPDR,PDMEDDT,PDMEDTM,NUPDRTM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,250377101,V,3403,V01,PDMEDUSE,10/2010,0,NaN,NaN,NaN,1,NaN,NaN,12:00:00,10/2010,2010-10-18 09:03:13.0,NaN,10/2010
1,250141601,V,3400,V01,PDMEDUSE,10/2010,0,NaN,NaN,NaN,1,NaN,NaN,10:30:00,10/2010,2010-10-15 12:42:19.0,NaN,10/2010
2,254095201,V,3406,V01,PDMEDUSE,11/2010,0,NaN,NaN,NaN,1,NaN,NaN,18:00:00,11/2010,2010-11-03 07:42:34.0,NaN,11/2010
3,263299301,V,3400,ST,PDMEDUSE,12/2010,0,NaN,NaN,NaN,1,NaN,NaN,11:15:00,12/2010,2010-12-14 13:49:21.0,NaN,12/2010
4,266425501,V,3407,V01,PDMEDUSE,01/2011,0,NaN,NaN,NaN,1,NaN,NaN,09:45:00,01/2011,2011-01-04 08:30:41.0,NaN,01/2011


### General_Physical_Exam: useful: Feat-Gen

In [166]:
GENPHYEX=dict_files[files_names[5]]
GENPHYEX.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PESEQ,PECAT,ABNORM,ABNORMCM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226890001,V,3402,SC,GENPHYEX,06/2010,3,Eyes,0.0,NaN,06/2010,2010-06-21 10:49:08.0,NaN,06/2010
1,226890301,V,3402,SC,GENPHYEX,06/2010,6,Cardiovascular (including peripheral vascular),0.0,NaN,06/2010,2010-06-21 10:49:08.0,NaN,06/2010
2,226890601,V,3402,SC,GENPHYEX,06/2010,9,Neurological,0.0,NaN,06/2010,2010-06-21 10:49:08.0,NaN,06/2010
3,232699801,V,3406,SC,GENPHYEX,07/2010,8,Musculoskeletal,0.0,NaN,07/2010,2010-07-14 07:53:01.0,NaN,07/2010
4,232700101,V,3406,SC,GENPHYEX,07/2010,11,Other (Specify location and describe),0.0,NaN,07/2010,2010-07-14 07:53:01.0,NaN,07/2010


### 'Concomitant_Medications.csv': Meds

In [169]:
CMED=dict_files[files_names[6]]
CMED.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,CMSEQ,CMTRT,CMDOSE,CMDOSU,CMDOSFRQ,...,LEDD,RECNO,SEQNO1,SEQNO2,WHODRUG,EXCLMED,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226846701,V,3400,LOG,CMED,1,LEXAPRO,20.0,MG,QD,...,NaN,15885.0,2.0,3.0,LEXAPRO,N,06/2010,2016-03-03 10:54:37.0,NaN,07/2010
1,226846801,V,3400,LOG,CMED,2,ALBUTEROL,2.0,PUFFS,PRN,...,NaN,1395.0,1.0,7.0,ALBUTEROL,N,06/2010,2012-07-13 05:32:46.0,NaN,07/2010
2,226846901,V,3400,LOG,CMED,3,BAUSCH & LOMB SOOTHE XL,2.0,GTTS,BID,...,NaN,28277.0,1.0,3.0,BAUSCH & LOMB MOISTURE EYES,N,06/2010,2010-10-20 06:22:16.0,NaN,07/2010
3,226847001,V,3400,LOG,CMED,4,MULTIVITIAMIN,1.0,TAB,QD,...,NaN,8317.0,1.0,1.0,MULTIVITAMIN,N,06/2010,2010-06-29 12:16:54.0,NaN,07/2010
4,226847101,V,3400,LOG,CMED,5,OMEGA 3 FISH OIL,1200.0,MG,QD,...,NaN,13341.0,1.0,8.0,OMEGA 3 FISH OIL,N,06/2010,2014-05-29 10:31:27.0,NaN,07/2010


### 'General_Neurological_Exam.csv': useful: Feat-Gen

In [171]:
PENEURO=dict_files[files_names[7]]
PENEURO.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,MSRARSP,MSRACM,MSLARSP,MSLACM,...,RFLLLRSP,RFLLLCM,PLRRRSP,PLRRCM,PLRLRSP,PLRLCM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232741801,V,3404,SC,PENEURO,06/2010,0,NaN,0,NaN,...,1.0,NaN,0.0,NaN,0.0,NaN,07/2010,2010-07-14 09:56:57.0,NaN,07/2010
1,232698601,V,3406,SC,PENEURO,07/2010,0,NaN,0,NaN,...,2.0,NaN,0.0,NaN,0.0,NaN,07/2010,2010-07-14 07:51:39.0,NaN,07/2010
2,226889601,V,3402,SC,PENEURO,06/2010,0,NaN,0,NaN,...,2.0,NaN,0.0,NaN,0.0,NaN,06/2010,2010-06-21 11:51:44.0,NaN,06/2010
3,226914401,V,3403,SC,PENEURO,06/2010,0,NaN,0,NaN,...,3.0,NaN,0.0,NaN,0.0,NaN,06/2010,2010-09-14 06:16:18.0,NaN,07/2010
4,226840201,V,3400,SC,PENEURO,06/2010,0,NaN,0,NaN,...,2.0,NaN,0.0,NaN,1.0,NaN,06/2010,2010-07-06 12:14:59.0,NaN,07/2010


### 'Current_Medical_Conditions_Log.csv': useful: Feat-Gen

In [173]:
CURRCOND=dict_files[files_names[8]]
CURRCOND.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,SEQNO,CONDCAT,DIAGYR,DXYREST,CONDTERM,...,LLT_NAME,PT_CODE,PT_NAME,SOCABBR1,VERBATIM,VMEDDRA,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,234853801,V,3404,LOG,CURRCOND,1,1P,2010.0,ACT,INCIDENTAL PINEAL CYST,...,Pineal gland cyst,10068650.0,Pineal gland cyst,Nerv,INCIDENTAL PINEAL CYST,16.1,07/2010,2017-08-07 11:40:05.0,NaN,07/2010
1,250714801,V,3400,LOG,CURRCOND,1,1E,2010.0,ACT,FLATTENING OF THE OPTIC DISCS (PER M.R.I),...,Optic disc disorder,10061321.0,Optic disc disorder,Eye,FLATTENING OF THE OPTIC DISCS (PER M.R.I),16.1,10/2010,2017-09-08 11:34:17.0,NaN,10/2010
2,250840101,V,3501,LOG,CURRCOND,1,1H,2000.0,ACT,HYPERTENSION,...,Hypertension,10020772.0,Hypertension,Vasc,HYPERTENSION,16.1,10/2010,2017-02-06 13:16:06.0,NaN,02/2011
3,255578501,V,3451,LOG,CURRCOND,1,1H,2000.0,YEAR,HYPERTENSION,...,Hypertension,10020772.0,Hypertension,Vasc,HYPERTENSION,16.1,11/2010,2018-04-12 09:24:59.0,NaN,12/2010
4,255578601,V,3451,LOG,CURRCOND,2,1D,1985.0,YEAR,VITILIGO HANDS AND LOWER EXTREMITIES,...,Vitiligo,10047642.0,Vitiligo,Skin,VITILIGO HANDS AND LOWER EXTREMITIES,16.1,11/2010,2018-04-12 09:24:59.0,NaN,12/2010


### 'Adverse_Event_Log.csv': useful: Meds
#### Measures the side effects of the meds taken for PD.

In [178]:
LOG=dict_files[files_names[9]]
LOG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,AESEQ,AETERM,STARTDT,STARTEST,STOPDT,...,PT_CODE,PT_NAME,HLT_NAME,HLGTNAME,SOCABBR1,VMEDDRA,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,237395401,V,3405,LOG,AE,1,POST LP HEADACHE,07/2010,ACT,08/2010,...,10019211.0,Post-LP Headache,Headaches NEC,Headaches,Nerv,16.1,08/2010,2016-12-14 08:36:35.0,NaN,08/2010
1,237395501,V,3405,LOG,AE,2,POST LP ABDOMINAL CRAMPS,07/2010,ACT,08/2010,...,10000081.0,Abdominal pain,Gastrointestinal and abdominal pains (excl ora...,Gastrointestinal signs and symptoms,Gastr,16.1,08/2010,2016-04-05 12:38:14.0,NaN,08/2010
2,241385701,V,3406,LOG,AE,1,LOSS OF CONSCIOUSNESS,08/2010,ACT,08/2010,...,10024855.0,Loss of consciousness,Disturbances in consciousness NEC,Neurological disorders NEC,Nerv,16.1,08/2010,2016-04-05 12:41:11.0,NaN,08/2010
3,250092501,V,3050,LOG,AE,1,MILD AND BRIEF UNILATERAL HEADACHE,09/2010,ACT,09/2010,...,10019211.0,Headache,Headaches NEC,Headaches,Nerv,16.1,10/2010,2016-04-05 12:38:12.0,NaN,NaN
4,251137901,V,3550,LOG,AE,1,METALLIC TASTE,10/2010,ACT,10/2010,...,10013911.0,Dysgeusia,Sensory abnormalities NEC,Neurological disorders NEC,Nerv,16.1,10/2010,2016-04-05 12:38:17.0,NaN,02/2011


### 'Initiation_of_PD_Medication-_incidents.csv' - useless

In [179]:
INCIDENTS=dict_files[files_names[10]]
INCIDENTS.head()

,PATNO,INITMD,INITMDDT,INITMDVS
0,3400,1,12/2010,V01
1,3403,1,01/2011,V01
2,3406,1,02/2011,V01
3,3502,1,05/2011,V01
4,3056,1,05/2011,V01


### 'Pregnancy_Form.csv' - very likely useless
#### could be useful if pregnancy means that patients cannot take some meds. 

### 'Surgery_for_Parkinson_Disease.csv': useful: Meds

In [181]:
SURGPD=dict_files[files_names[12]]
SURGPD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PDSURG,PDSURGDT,PDSRGEST,PDSURGTP,...,PDSLSTN,PDSLOTH,PDSLOTCM,PDSLNA,PDSLUNK,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,517638901,V,41380,V02,SURGPD,04/2015,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,04/2015,2015-04-28 03:49:56.0,NaN,04/2015
1,517719901,V,4076,V07,SURGPD,03/2015,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,04/2015,2015-04-28 12:00:19.0,NaN,04/2015
2,517792201,V,3914,V06,SURGPD,04/2015,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,04/2015,2015-04-29 02:55:53.0,NaN,04/2015
3,517800901,V,3904,V07,SURGPD,04/2015,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,04/2015,2015-04-29 05:32:48.0,NaN,04/2015
4,518003501,V,40733,SC,SURGPD,04/2015,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,04/2015,2015-04-30 00:09:32.0,NaN,04/2015


### 'Neurological_Exam_-_Cranial_Nerves.csv': useful: Feat-PD

In [183]:
PENEURO=dict_files[files_names[13]]
PENEURO.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,CN1RSP,CN1CM,CN2RSP,CN2CM,...,CN910RSP,CN910CM,CN11RSP,CN11CM,CN12RSP,CN12CM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232741701,V,3404,SC,PENEURO,06/2010,0.0,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2010-07-14 09:56:57.0,NaN,07/2010
1,232698501,V,3406,SC,PENEURO,07/2010,0.0,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2010-07-14 07:51:39.0,NaN,07/2010
2,226889501,V,3402,SC,PENEURO,06/2010,0.0,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,06/2010,2010-06-21 10:48:26.0,NaN,06/2010
3,226914301,V,3403,SC,PENEURO,06/2010,0.0,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,06/2010,2010-06-21 11:18:43.0,NaN,07/2010
4,226840101,V,3400,SC,PENEURO,06/2010,0.0,NaN,0.0,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,06/2010,2010-06-21 08:48:00.0,NaN,07/2010


### 'PD_Features.csv': useful: Feat-PD

In [185]:
PDFEAT=dict_files[files_names[14]]
PDFEAT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,SXMO,SXYEAR,PDDXDT,PDDXEST,...,DXRIGID,DXBRADY,DXPOSINS,DXOTHSX,DXOTHCM,DOMSIDE,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226923001,V,3403,SC,PDFEAT,06/2010,6.0,2009,03/2010,DAY,...,1,0,0,0,NaN,2,06/2010,2010-11-23 11:43:25.0,NaN,07/2010
1,226845501,V,3400,SC,PDFEAT,06/2010,3.0,2007,10/2009,ACT,...,1,1,0,0,NaN,1,06/2010,2010-09-13 15:27:03.0,NaN,07/2010
2,226895901,V,3402,SC,PDFEAT,06/2010,7.0,2008,03/2010,DAY,...,0,1,0,0,NaN,2,06/2010,2010-06-21 10:54:56.0,NaN,06/2010
3,232711801,V,3406,SC,PDFEAT,07/2010,6.0,2008,01/2010,DAY,...,1,1,0,0,NaN,1,07/2010,2012-11-07 07:37:56.0,NaN,07/2010
4,241672401,V,3407,SC,PDFEAT,08/2010,1.0,2010,02/2010,DAY,...,1,1,0,0,NaN,1,09/2010,2010-09-01 06:21:36.0,NaN,09/2010


### 'Vital_Signs.csv': useful: Feat-Gen

In [188]:
VITAL=dict_files[files_names[15]]
VITAL.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,WGTKG,HTCM,TEMPC,BPARM,...,DIASUP,HRSUP,SYSSTND,DIASTND,HRSTND,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,408779301,V,10362,SC,VITAL,08/2013,NaN,NaN,36.0,1.0,...,65.0,78.0,114.0,71.0,77.0,NaN,08/2013,2013-08-23 11:43:57.0,NaN,08/2013
1,605754701,V,10405,SC,VITAL,09/2014,NaN,NaN,36.9,1.0,...,81.0,64.0,150.0,80.0,73.0,Subject notes that this is higher than her nor...,12/2016,2016-12-06 13:51:42.0,NaN,12/2016
2,411032401,V,10496,SC,VITAL,06/2013,NaN,NaN,36.1,2.0,...,90.0,93.0,176.0,86.0,98.0,NaN,09/2013,2013-09-05 13:27:37.0,NaN,09/2013
3,409587001,S,10606,SC,VITAL,08/2013,NaN,NaN,36.6,2.0,...,92.0,51.0,170.0,61.0,58.0,subject states BP usually high,09/2013,2015-06-26 08:23:43.0,NaN,03/2015
4,448228101,V,10662,SC,VITAL,07/2014,NaN,NaN,37.5,2.0,...,80.0,76.0,130.0,78.0,80.0,NaN,07/2014,2014-07-15 11:04:09.0,NaN,07/2014


## 6-Motor-Assessments - all of them can be useful. 
#### But we might make the choice to not take all of them. 
#### For assessing the disease state (the labels), I think we should focus on the UPDRS1,3,&4 and take the other assessments as features

In [190]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/6-Motor-Assessments/'
# saving all the csv files in a dataframe
import os
import glob
# glob.glob('*.csv') #find all the csv files in a pathname. 
os.chdir(folder_path)
csv_files = [i for i in glob.glob('*.csv')]
csv_files

# Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
dict_files={}
files_names=[]
for files in csv_files:
#for i, files in enumerate(csv_files): 
    df=pd.read_csv(files,engine='python')
    basepath, filename=ntpath.split(files)
    files_names.append(filename)
    dict_files[filename]=df


# Calculating the # of rows and features for each dataframe and storing them in a list of tuples
shape_df=[]
for i in range(0,len(dict_files.keys())):
    nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
    shape_df.append((nrows,ncols))
print(shape_df)
print(list(dict_files.keys()))

# DF TO EXCEL
#from pandas import ExcelWriter

#writer = ExcelWriter('PythonExport.xlsx')
#yourdf.to_excel(writer,'Sheet5')
#writer.save()

[(11234, 18), (32, 15), (11236, 17), (3903, 18), (75, 60), (4541, 16), (10191, 11), (12696, 51), (295, 18), (503, 20), (23430, 13), (11235, 24)]
['MDS_UPDRS_Part_I__Patient_Questionnaire.csv', 'TAP-PD_OPDM_Use_Questionnaire.csv', 'MDS_UPDRS_Part_I.csv', 'PASE_-_Household_Activity.csv', 'Gait_Data___Arm_swing.csv', 'MDS_UPDRS_Part_IV.csv', 'Modified_Schwab_+_England_ADL.csv', 'MDS_UPDRS_Part_III.csv', 'TAP-PD_OPDM_Assessment.csv', 'TAP-PD_Kinetics_Device_Testing.csv', 'PASE_-_Leisure_Time_Activity.csv', 'MDS_UPDRS_Part_II__Patient_Questionnaire.csv']


### 'MDS_UPDRS_Part_I__Patient_Questionnaire.csv'

In [205]:
NUPDR1SP=dict_files[files_names[0]]
NUPDR1SP.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP1SLPN,NP1SLPD,NP1PAIN,NP1URIN,NP1CNST,NP1LTHD,NP1FATG,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232593201,V,3400,SC,NUPDRS1P,07/2010,1.0,1.0,1.0,3.0,1.0,2.0,1.0,3.0,07/2010,2010-07-13 16:42:58.0,NaN,07/2010
1,230957601,V,3403,SC,NUPDRS1P,07/2010,1.0,0.0,1.0,1.0,0.0,3.0,0.0,2.0,07/2010,2010-07-06 13:08:13.0,NaN,07/2010
2,233164501,V,3400,BL,NUPDRS1P,07/2010,1.0,0.0,1.0,2.0,1.0,1.0,1.0,4.0,07/2010,2010-07-15 17:09:39.0,NaN,07/2010
3,234837801,V,3404,BL,NUPDRS1P,07/2010,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,07/2010,2010-07-23 11:35:18.0,NaN,07/2010
4,232529501,V,3401,BL,NUPDRS1P,07/2010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-13 13:36:12.0,NaN,07/2010


### 'TAP-PD_OPDM_Use_Questionnaire.csv'

In [207]:
TAPUSE=dict_files[files_names[1]]
TAPUSE.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,UNDRSDIR,CONFCORR,FITSCHED,NDREMIND,AFFPPMI,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,328345501,V,3432,V01,TAPUSE,01/2012,0,3,0,0,2,01/2012,2012-01-05 07:27:06.0,NaN,01/2012
1,329150101,V,3433,V01,TAPUSE,01/2012,0,3,0,0,2,01/2012,2012-01-12 08:45:59.0,NaN,01/2012
2,329429701,V,3435,V01,TAPUSE,01/2012,0,3,0,0,2,01/2012,2012-01-14 16:39:59.0,NaN,01/2012
3,332894601,V,3218,V01,TAPUSE,02/2012,0,3,0,0,2,02/2012,2012-02-08 11:51:38.0,NaN,02/2012
4,335013101,V,3436,V01,TAPUSE,02/2012,0,3,0,1,2,02/2012,2012-02-23 13:12:10.0,NaN,02/2012


### 'MDS_UPDRS_Part_I.csv'

In [210]:
NUPDRS1=dict_files[files_names[2]]
NUPDRS1.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,NP1DDS,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,231798601,V,3402,SC,NUPDRS1,07/2010,1.0,0.0,0.0,1.0,0.0,2.0,0.0,07/2010,2010-07-09 12:00:30.0,NaN,07/2010
1,233164301,V,3400,BL,NUPDRS1,07/2010,1.0,0.0,0.0,2.0,3.0,2.0,0.0,07/2010,2010-07-15 17:04:33.0,NaN,07/2010
2,230955201,V,3403,SC,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-06 13:03:54.0,NaN,07/2010
3,234837501,V,3404,BL,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-23 11:30:12.0,NaN,07/2010
4,232712901,V,3406,SC,NUPDRS1,07/2010,1.0,0.0,0.0,0.0,1.0,1.0,0.0,07/2010,2010-07-14 08:54:29.0,NaN,07/2010


### 'PASE_-_Household_Activity.csv'

In [214]:
PASE=dict_files[files_names[3]]
PASE.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,LTHSWRK,HVYHSWRK,HMREPR,LAWNWRK,OUTGARDN,CAREGVR,WRKVL,WRKVLHR,WRKVLACT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,456171801,V,10874,BL,PASE,2,1,1.0,2,2.0,1.0,1.0,NaN,NaN,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
1,531597501,V,10874,V04,PASE,2,2,1.0,2,2.0,1.0,1.0,NaN,NaN,08/2015,2015-08-17 08:54:53.0,NaN,2015-08-17 08:55:04.0
2,585712501,V,10874,V06,PASE,2,2,1.0,2,2.0,1.0,1.0,NaN,NaN,08/2016,2016-08-24 11:52:50.0,NaN,2016-08-24 11:53:02.0
3,648651901,V,10874,V08,PASE,2,2,1.0,2,2.0,2.0,1.0,NaN,NaN,08/2017,2017-08-18 12:07:35.0,NaN,2017-08-18 12:11:04.0
4,416212301,V,12224,BL,PASE,2,2,1.0,2,2.0,1.0,2.0,2.0,2.0,10/2013,2013-10-30 12:23:09.0,NaN,2013-10-30 12:23:19.0


### 'Gait_Data___Arm_swing.csv'

In [216]:
PASE=dict_files[files_names[4]]
PASE.head()

,PATNO,EVENT_ID,INFODT,COHORT,SP_U,RA_AMP_U,LA_AMP_U,RA_STD_U,LA_STD_U,SYM_U,...,TUG1_STRAIGHT_DUR,TUG1_TURNS_DUR,TUG1_STEP_REG,TUG1_STEP_SYM,TUG2_DUR,TUG2_STEP_NUM,TUG2_STRAIGHT_DUR,TUG2_TURNS_DUR,TUG2_STEP_REG,TUG2_STEP_SYM
0,42415,BL,12/2016,3.0,0.995,20.253311,25.345497,2.601230,3.655499,0.198092,...,0.570312,2.699219,0.694346,1.503600,10.796875,8.163086,0.546875,2.453125,0.598333,1.783785
1,41568,V06,06/2017,3.0,1.138,12.457885,31.332200,2.235030,3.304857,0.598351,...,0.601562,2.695312,0.292028,0.735896,11.648438,10.804688,0.601562,2.023438,0.376570,1.858731
2,42450,V04,01/2018,1.0,0.994,30.585239,38.063166,5.048072,3.617321,0.194228,...,0.585938,1.218750,0.590433,1.071284,10.070312,11.744141,0.570312,1.113281,0.664089,1.053335
3,42415,V04,01/2018,3.0,0.996,17.191551,29.273684,2.702083,3.963163,0.412973,...,0.515625,2.039062,0.598600,1.678548,10.492188,11.323242,0.515625,1.742188,0.761910,1.146845
4,42357,V04,01/2018,3.0,0.881,7.695238,3.838619,2.113256,1.966532,1.096781,...,0.523438,2.222656,0.596043,1.098940,11.195312,16.933594,0.468750,1.539062,0.509521,1.191527


### 'MDS_UPDRS_Part_IV.csv'

In [219]:
NUPDRS4=dict_files[files_names[5]]
NUPDRS4.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,NP4WDYSK,NP4DYSKI,NP4OFF,NP4FLCTI,NP4FLCTX,NP4DYSTN,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,267411701,V,3403,ST,NUPDRS4,01/2011,0.0,0.0,0.0,0.0,0.0,0.0,01/2011,2011-01-08 09:34:03.0,NaN,01/2011
1,280733601,V,3251,U01,NUPDRS4,02/2011,0.0,0.0,0.0,0.0,0.0,0.0,03/2011,2011-03-16 12:19:26.0,NaN,03/2011
2,304301401,V,3800,ST,NUPDRS4,07/2011,0.0,0.0,0.0,0.0,0.0,0.0,07/2011,2011-07-11 05:07:01.0,NaN,07/2011
3,303916001,V,3400,V04,NUPDRS4,07/2011,0.0,0.0,0.0,0.0,0.0,0.0,07/2011,2011-07-07 08:06:50.0,NaN,07/2011
4,306573401,V,3403,V04,NUPDRS4,07/2011,0.0,0.0,0.0,0.0,0.0,0.0,07/2011,2011-07-27 08:20:55.0,NaN,07/2011


### 'Modified_Schwab_+_England_ADL.csv'

In [223]:
MODSEADL=dict_files[files_names[6]]
MODSEADL.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,MSEADLG,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,230954401,V,3403,SC,MODSEADL,06/2010,90.0,07/2010,2010-07-06 13:02:56.0,NaN,07/2010
1,229763901,V,3402,SC,MODSEADL,06/2010,95.0,06/2010,2010-06-30 13:49:58.0,NaN,06/2010
2,230939601,V,3400,SC,MODSEADL,06/2010,90.0,07/2010,2010-07-06 12:36:23.0,NaN,07/2010
3,233166301,V,3400,BL,MODSEADL,07/2010,90.0,07/2010,2010-07-15 17:20:22.0,NaN,07/2010
4,235635801,V,3406,SC,MODSEADL,07/2010,90.0,07/2010,2010-07-28 12:02:16.0,NaN,07/2010


### 'MDS_UPDRS_Part_III.csv'

In [227]:
NUPDRS3=dict_files[files_names[7]]
NUPDRS3.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,CMEDTM,EXAMTM,NP3SPCH,NP3FACXP,...,DYSKPRES,DYSKIRAT,NHY,ANNUAL_TIME_BTW_DOSE_NUPDRS,ON_OFF_DOSE,PD_MED_USE,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,324141901,V,3506,V03,NUPDRS3,10/2011,NaN,NaN,1.0,1.0,...,0.0,NaN,1.0,NaN,NaN,0,12/2011,2011-12-05 19:23:15.0,NaN,12/2011
1,324180301,V,3419,V02,NUPDRS3,12/2011,NaN,NaN,0.0,0.0,...,0.0,NaN,1.0,NaN,NaN,0,12/2011,2011-12-06 07:25:10.0,NaN,12/2011
2,324389901,V,3816,BL,NUPDRS3,12/2011,NaN,NaN,0.0,0.0,...,0.0,NaN,0.0,NaN,NaN,0,12/2011,2011-12-07 02:10:18.0,NaN,12/2011
3,324396501,V,3762,BL,NUPDRS3,11/2011,NaN,NaN,0.0,1.0,...,1.0,1.0,2.0,NaN,NaN,0,12/2011,2011-12-07 02:34:41.0,NaN,12/2011
4,324399901,V,3765,BL,NUPDRS3,11/2011,NaN,NaN,0.0,0.0,...,0.0,NaN,0.0,NaN,NaN,0,12/2011,2011-12-07 02:47:42.0,NaN,12/2011


### 'TAP-PD_OPDM_Assessment.csv'

In [230]:
TAPOPDM=dict_files[files_names[8]]
TAPOPDM.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,OPDMCMPL,DEVICESN,DIGITOGR,PACKEYBD,PEGBRD,XFRYN,XFRDT,OPDMTM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,312776601,V,3432,BL,TAPOPDM,09/2011,1,HVDG9341016,1.0,1.0,1.0,1.0,09/2011,NaN,09/2011,2011-11-18 13:02:26.0,NaN,09/2011
1,314756601,V,3433,BL,TAPOPDM,09/2011,1,HVDG9341006,1.0,1.0,1.0,1.0,09/2011,NaN,09/2011,2011-11-18 13:00:24.0,NaN,09/2011
2,318524501,V,3435,BL,TAPOPDM,10/2011,1,HVDG9353016,1.0,1.0,1.0,1.0,10/2011,NaN,10/2011,2011-11-18 13:03:51.0,NaN,10/2011
3,322801201,V,3436,BL,TAPOPDM,11/2011,1,HVDG9343009,1.0,1.0,1.0,1.0,11/2011,NaN,12/2011,2011-12-01 10:03:16.0,NaN,12/2011
4,325919401,V,3220,BL,TAPOPDM,12/2011,1,HVDG9341003,1.0,1.0,1.0,0.0,NaN,NaN,12/2011,2011-12-10 13:19:37.0,NaN,12/2011


### 'TAP-PD_Kinetics_Device_Testing.csv'

In [233]:
KINETICS=dict_files[files_names[9]]
KINETICS.head()

,PATNO,TESTDATE,TAPDAY,REVISION,DEVICESN,QMAT_OMS,QMAT_KEYDOWN,QMAT_KEYDOWN_RIGHT,QMAT_KEYDOWN_LEFT,QMAT_PEGCYCLE,QMAT_PEGCYCLE_RIGHT,QMAT_PEGCYCLE_LEFT,QMAT_KEYTRAN,QMAT_KEYTRAN_RIGHT,QMAT_KEYTRAN_LEFT,DOM_SIDE,AFF_SIDE,SEX,CONTROL,NOTES
0,3432,09/2011,0.0,R4,HVDG9341016,40.462,3.717,3.633,3.793,0.808,0.844,0.770,-0.980,-1.372,-0.699,NaN,NaN,Male,False,"Pegboard test, right hand--Some pegs were remo..."
1,3432,09/2011,7.0,R4,HVDG9341016,41.180,3.590,3.495,3.677,0.750,0.785,0.713,-0.981,-1.091,-0.883,NaN,NaN,Male,False,NaN
2,3432,09/2011,14.0,R4,HVDG9341016,41.975,3.506,3.356,3.636,0.704,0.715,0.694,-0.904,-1.017,-0.802,NaN,NaN,Male,False,"Pegboard test, right hand--Some pegs were remo..."
3,3435,10/2011,0.0,R4,HVDG9353016,46.176,3.765,3.438,4.012,1.002,1.072,0.927,-0.417,-0.144,-0.794,NaN,NaN,NaN,False,NaN
4,3218,10/2011,0.0,R4,HVDG9341013,39.231,4.139,4.464,3.655,0.941,0.809,1.057,-0.578,-0.697,-0.472,Right,Left,Male,False,NaN


### 'PASE_-_Leisure_Time_Activity.csv'

In [236]:
PASE=dict_files[files_names[10]]
PASE.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,QUESTNO,ACTVOFT,ACTVSPEC,HRDAYFRQ,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,456171201,V,10874,BL,PASE,1,3.0,"reading, computer, TV",3.0,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
1,456171301,V,10874,BL,PASE,2,3.0,NaN,2.0,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
2,456171401,V,10874,BL,PASE,3,3.0,gardening,3.0,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
3,456171501,V,10874,BL,PASE,4,0.0,NaN,NaN,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0
4,456171601,V,10874,BL,PASE,5,0.0,NaN,NaN,09/2014,2014-09-11 09:49:22.0,NaN,2014-09-11 09:49:33.0


### 'MDS_UPDRS_Part_II__Patient_Questionnaire.csv'

In [241]:
NUPDRS2P=dict_files[files_names[11]]
NUPDRS2P.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,NUPSOURC,NP2SPCH,NP2SALV,NP2SWAL,...,NP2HOBB,NP2TURN,NP2TRMR,NP2RISE,NP2WALK,NP2FREZ,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232529901,V,3401,BL,NUPDRS2P,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-13 13:37:01.0,NaN,07/2010
1,232713901,V,3406,SC,NUPDRS2P,07/2010,1.0,0.0,2.0,0.0,...,2.0,0.0,3.0,0.0,0.0,0.0,07/2010,2010-07-14 08:56:34.0,NaN,07/2010
2,231800401,V,3402,SC,NUPDRS2P,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,07/2010,2010-07-09 12:04:22.0,NaN,07/2010
3,232595801,V,3400,SC,NUPDRS2P,07/2010,1.0,0.0,2.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,07/2010,2010-07-13 16:48:22.0,NaN,07/2010
4,230957201,V,3403,SC,NUPDRS2P,07/2010,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,0.0,07/2010,2010-07-06 13:06:55.0,NaN,07/2010


## 7-Non-motor-assessments - all of them can be useful but we might need to choose. 
### Do a correlation analysis and ask doctors which ones are the more relevant. 

In [242]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/7-Non-motor-assessments/'
# saving all the csv files in a dataframe
import os
import glob
# glob.glob('*.csv') #find all the csv files in a pathname. 
os.chdir(folder_path)
csv_files = [i for i in glob.glob('*.csv')]
csv_files

# Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
dict_files={}
files_names=[]
for files in csv_files:
#for i, files in enumerate(csv_files): 
    df=pd.read_csv(files,engine='python')
    basepath, filename=ntpath.split(files)
    files_names.append(filename)
    dict_files[filename]=df


# Calculating the # of rows and features for each dataframe and storing them in a list of tuples
shape_df=[]
for i in range(0,len(dict_files.keys())):
    nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
    shape_df.append((nrows,ncols))
print(shape_df)
print(list(dict_files.keys()))

# DF TO EXCEL
#from pandas import ExcelWriter

#writer = ExcelWriter('PythonExport.xlsx')
#yourdf.to_excel(writer,'Sheet5')
#writer.save()

[(1992, 16), (80, 21), (6353, 19), (5300, 34), (4668, 85), (6982, 33), (2760, 17), (6356, 46), (5152, 16), (5309, 24), (6186, 38), (5307, 16), (6640, 25), (5298, 16), (6627, 50), (5294, 46), (6353, 24)]
['University_of_Pennsylvania_Smell_ID_Test.csv', 'Features_of_REM_Behavior_Disorder.csv', 'Epworth_Sleepiness_Scale.csv', 'Letter_-_Number_Sequencing__PD_.csv', 'Olfactory_UPSIT.csv', 'REM_Sleep_Disorder_Questionnaire.csv', 'Cognitive_Assessments.csv', 'SCOPA-AUT.csv', 'Cognitive_Categorization.csv', 'Hopkins_Verbal_Learning_Test.csv', 'Montreal_Cognitive_Assessment__MoCA_.csv', 'Semantic_Fluency.csv', 'Geriatric_Depression_Scale__Short_.csv', 'Symbol_Digit_Modalities.csv', 'State-Trait_Anxiety_Inventory.csv', 'Benton_Judgment_of_Line_Orientation.csv', 'QUIP_Current_Short.csv']


### 'University_of_Pennsylvania_Smell_ID_Test.csv'

In [245]:
UPSIT=dict_files[files_names[0]]
UPSIT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,UPSITBK1,UPSITBK2,UPSITBK3,UPSITBK4,NORMATIVE_SCORE,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,231754301,V,3403,BL,UPSIT,07/2010,4.0,2.0,1.0,2.0,Anosmia,NaN,07/2010,2010-07-09 09:53:39.0,NaN,07/2010
1,232551901,V,3401,BL,UPSIT,07/2010,9.0,9.0,5.0,4.0,Hyposmia,NaN,07/2010,2010-07-13 14:23:31.0,NaN,07/2010
2,233828501,V,3400,BL,UPSIT,07/2010,6.0,8.0,7.0,9.0,Hyposmia,NaN,07/2010,2010-07-20 07:37:29.0,NaN,07/2010
3,234847101,V,3404,BL,UPSIT,07/2010,8.0,6.0,9.0,10.0,Hyposmia,NaN,07/2010,2010-07-23 12:15:13.0,NaN,07/2010
4,236093901,V,3405,BL,UPSIT,07/2010,9.0,10.0,10.0,10.0,Normosmia,NaN,07/2010,2010-07-30 10:34:31.0,NaN,07/2010


### 'Features_of_REM_Behavior_Disorder.csv'

In [246]:
UPSIT=dict_files[files_names[1]]
UPSIT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,REMONSDT,REMONEST,RBDDXDT,RBDDXEST,...,ONBENZ,ONMLATON,ONSSRI,ONNORSRI,ONTRIADP,ONBTABLK,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,608680701,V,60048,SC,REMBHVDS,06/2014,01/1987,2.0,07/2007,1,...,0.0,0.0,0.0,0.0,0.0,0.0,12/2016,2016-12-21 08:10:43.0,NaN,12/2016
1,608900501,V,60033,SC,REMBHVDS,12/2016,06/1996,3.0,05/2014,1,...,0.0,1.0,1.0,1.0,0.0,0.0,12/2016,2016-12-22 09:13:32.0,NaN,12/2016
2,609531601,V,60065,SC,REMBHVDS,03/2014,01/2005,3.0,01/2010,1,...,0.0,0.0,0.0,0.0,0.0,0.0,12/2016,2016-12-29 04:02:29.0,NaN,12/2016
3,609625701,V,60146,SC,REMBHVDS,12/2014,01/2007,3.0,04/2011,1,...,0.0,0.0,0.0,0.0,0.0,0.0,12/2016,2016-12-30 01:06:15.0,NaN,12/2016
4,609524901,V,60059,SC,REMBHVDS,02/2014,12/2002,3.0,02/2011,1,...,0.0,0.0,0.0,0.0,0.0,0.0,12/2016,2016-12-28 23:53:04.0,NaN,12/2016


### 'Epworth_Sleepiness_Scale.csv

In [251]:
EPWORTH=dict_files[files_names[2]]
EPWORTH.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTCGBOTH,ESS1,ESS2,ESS3,ESS4,ESS5,ESS6,ESS7,ESS8,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,234842801,V,3404,BL,EPWORTH,07/2010,1.0,1.0,1.0,1.0,0.0,3.0,0.0,1.0,0.0,07/2010,2010-07-23 11:54:04.0,NaN,07/2010
1,233819801,V,3400,BL,EPWORTH,07/2010,1.0,2.0,2.0,1.0,3.0,3.0,0.0,1.0,0.0,07/2010,2010-07-20 07:17:01.0,NaN,07/2010
2,232533701,V,3401,BL,EPWORTH,07/2010,1.0,1.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,07/2010,2010-07-13 13:44:34.0,NaN,07/2010
3,236092201,V,3405,BL,EPWORTH,07/2010,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,07/2010,2010-07-30 10:24:35.0,NaN,07/2010
4,231743201,V,3403,BL,EPWORTH,07/2010,1.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,07/2010,2010-07-09 09:39:50.0,NaN,07/2010


### 'Letter_-_Number_Sequencing__PD_.csv'

In [252]:
LNSPD=dict_files[files_names[3]]
LNSPD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,LNS1A,LNS1B,LNS1C,LNS2A,...,LNS7A,LNS7B,LNS7C,LNS_TOTRAW,AGE_ASSESS_LNS,DVS_LNS,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233165701,V,3400,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10.0,39.0,9.0,07/2010,2010-07-15 17:16:27.0,NaN,07/2010
1,232532701,V,3401,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10.0,56.0,11.0,07/2010,2010-07-13 13:43:06.0,NaN,07/2010
2,234840901,V,3404,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,15.0,55.0,16.0,07/2010,2010-07-23 11:51:30.0,NaN,07/2010
3,231742701,V,3403,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,10.0,68.0,11.0,07/2010,2010-07-09 09:38:07.0,NaN,07/2010
4,236091801,V,3405,BL,LNSPD,07/2010,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,9.0,63.0,10.0,07/2010,2010-07-30 10:21:29.0,NaN,07/2010


### 'Olfactory_UPSIT.csv'

In [249]:
OLFACT=dict_files[files_names[4]]
OLFACT.head()

,SUBJECT_ID,PROTOCOL_ID,COMPLT_DATE,SCENT_10_RESPONSE,SCENT_09_RESPONSE,SCENT_08_RESPONSE,SCENT_07_RESPONSE,SCENT_06_RESPONSE,SCENT_05_RESPONSE,SCENT_04_RESPONSE,...,SCENT_38_CORRECT,SCENT_37_CORRECT,SCENT_36_CORRECT,SCENT_35_CORRECT,SCENT_34_CORRECT,SCENT_33_CORRECT,SCENT_32_CORRECT,SCENT_31_CORRECT,TOTAL_CORRECT,UPSIIT_PRCNTGE
0,90000,P-PPMI Cohort,04/2013,2,3,2,1,2,3,4,...,1,1,0,1,1,1,1,1,38,98
1,90001,P-PPMI Cohort,04/2013,2,3,2,1,2,3,4,...,1,0,1,1,0,1,0,1,30,36
2,90002,P-PPMI Cohort,03/2013,2,3,2,1,2,1,4,...,1,1,1,1,0,1,1,1,28,36
3,90003,P-PPMI Cohort,04/2013,3,1,1,4,2,4,3,...,0,0,0,0,0,0,0,0,8,0
4,90004,P-PPMI Cohort,04/2013,1,3,2,1,2,3,4,...,1,1,0,1,1,1,0,1,32,75


### 'REM_Sleep_Disorder_Questionnaire.csv'

In [254]:
REMSLEEP=dict_files[files_names[5]]
REMSLEEP.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTCGBOTH,DRMVIVID,DRMAGRAC,DRMNOCTB,...,NARCLPSY,DEPRS,EPILEPSY,BRNINFM,CNSOTH,CNSOTHCM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232534301,V,3401,BL,REMSLEEP,07/2010,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-13 13:46:12.0,NaN,07/2010
1,233821601,V,3400,BL,REMSLEEP,07/2010,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,NaN,07/2010,2010-07-20 07:19:50.0,NaN,07/2010
2,234843201,V,3404,BL,REMSLEEP,07/2010,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-23 11:55:28.0,NaN,07/2010
3,231743401,V,3403,BL,REMSLEEP,07/2010,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-09 09:41:25.0,NaN,07/2010
4,236092401,V,3405,BL,REMSLEEP,07/2010,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,07/2010,2010-07-30 10:27:21.0,NaN,07/2010


### 'Cognitive_Assessments.csv'

In [257]:
COGTIME=dict_files[files_names[6]]
COGTIME.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,HVLTRTTM,HVLTDRTM,LNORNTTM,SFTTM,LNSTM,SDMTM,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,517792401,V,3914,V06,COGTIME,04/2015,11:18:00,11:45:00,NaN,NaN,NaN,NaN,Only noted times for HVLT-R during this visit....,04/2015,2015-04-29 03:01:42.0,NaN,04/2015
1,518015001,V,3871,V06,COGTIME,04/2015,NaN,NaN,NaN,NaN,NaN,NaN,Not completed- test introduced only very recen...,04/2015,2015-04-30 01:38:31.0,NaN,05/2015
2,518059301,V,41291,BL,COGTIME,04/2015,18:00:00,18:20:00,18:25:00,18:30:00,18:33:00,18:43:00,NaN,04/2015,2015-04-30 06:26:32.0,NaN,04/2015
3,520658401,V,4114,V06,COGTIME,05/2015,14:58:00,15:38:00,15:12:00,15:18:00,15:26:00,15:32:00,NaN,05/2015,2015-05-05 08:43:23.0,NaN,05/2015
4,520722501,V,3446,V08,COGTIME,05/2015,09:58:00,10:17:00,10:03:00,10:05:00,10:09:00,10:12:00,NaN,05/2015,2015-05-05 12:00:25.0,NaN,05/2015


### 'SCOPA-AUT.csv'

In [260]:
SCOPAAUT=dict_files[files_names[7]]
SCOPAAUT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTCGBOTH,SCAU1,SCAU2,SCAU3,...,SCAU26B,SCAU26BT,SCAU26C,SCAU26CT,SCAU26D,SCAU26DT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233827401,V,3400,BL,SCOPAAUT,07/2010,1.0,0.0,1.0,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2012-01-08 08:13:13.0,NaN,07/2010
1,232551401,V,3401,BL,SCOPAAUT,07/2010,1.0,0.0,0.0,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2012-01-08 08:13:14.0,NaN,07/2010
2,234844601,V,3404,BL,SCOPAAUT,07/2010,1.0,0.0,0.0,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2012-01-08 08:13:14.0,NaN,07/2010
3,236093701,V,3405,BL,SCOPAAUT,07/2010,1.0,0.0,0.0,0.0,...,0.0,NaN,0.0,NaN,0.0,NaN,07/2010,2012-01-08 08:13:14.0,NaN,07/2010
4,231753901,V,3403,BL,SCOPAAUT,07/2010,1.0,0.0,0.0,0.0,...,0.0,NaN,1.0,Enalapril,0.0,NaN,07/2010,2012-01-08 08:13:14.0,NaN,07/2010


### Cognitive_Categorization.csv'

In [263]:
COGCATG=dict_files[files_names[8]]
COGCATG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTCGBOTH,COGDECLN,FNCDTCOG,COGSTATE,COGDXCL,RVWNPSY,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,365358701,V,3574,ST,COGCATG,10/2012,1.0,0.0,0.0,1.0,1.0,0.0,10/2012,2012-10-04 09:02:01.0,NaN,10/2012
1,355330201,V,3179,BL,COGCATG,07/2012,3.0,0.0,0.0,1.0,1.0,1.0,07/2012,2012-07-23 17:22:03.0,NaN,07/2012
2,351869301,V,3178,BL,COGCATG,06/2012,1.0,1.0,0.0,2.0,2.0,1.0,06/2012,2012-06-25 18:20:31.0,NaN,06/2012
3,362798601,V,3432,ST,COGCATG,08/2012,1.0,0.0,0.0,1.0,1.0,0.0,09/2012,2012-09-18 13:44:55.0,NaN,09/2012
4,364318401,V,3407,V06,COGCATG,09/2012,1.0,0.0,0.0,1.0,1.0,0.0,09/2012,2012-09-27 12:15:21.0,NaN,09/2012


### 'Hopkins_Verbal_Learning_Test.csv'

In [265]:
HVLT=dict_files[files_names[9]]
HVLT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,HVLTRT1,HVLTRT2,HVLTRT3,HVLTRDLY,...,comm,AGE_ASSESS_HVLT,DVT_TOTAL_RECALL,DVT_DELAYED_RECALL,DVT_RETENTION,DVT_RECOG_DISC_INDEX,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,231741801,V,3403,BL,HVLT,07/2010,6.0,7.0,8.0,9.0,...,NaN,68.0,44.0,52.0,63.0,47.0,07/2010,2012-06-03 09:05:07.0,NaN,07/2010
1,232531001,V,3401,BL,HVLT,07/2010,6.0,10.0,11.0,12.0,...,NaN,56.0,49.0,61.0,62.0,51.0,07/2010,2012-06-03 09:05:07.0,NaN,07/2010
2,233165101,V,3400,BL,HVLT,07/2010,8.0,10.0,10.0,11.0,...,NaN,39.0,49.0,54.0,62.0,58.0,07/2010,2012-06-03 09:05:07.0,NaN,07/2010
3,234838901,V,3404,BL,HVLT,07/2010,8.0,10.0,11.0,12.0,...,NaN,55.0,54.0,61.0,62.0,44.0,07/2010,2012-06-03 09:05:07.0,NaN,07/2010
4,236090801,V,3405,BL,HVLT,07/2010,6.0,9.0,12.0,10.0,...,NaN,63.0,51.0,53.0,44.0,52.0,07/2010,2012-06-03 09:05:08.0,NaN,07/2010


### 'Montreal_Cognitive_Assessment__MoCA_.csv'

In [267]:
MOCA=dict_files[files_names[10]]
MOCA.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,MCAALTTM,MCACUBE,MCACLCKC,MCACLCKN,...,MCAMONTH,MCAYR,MCADAY,MCAPLACE,MCACITY,MCATOT,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,226919901,V,3403,SC,MOCA,06/2010,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,25.0,06/2010,2010-06-21 11:25:04.0,NaN,07/2010
1,232509801,V,3401,SC,MOCA,07/2010,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,28.0,07/2010,2010-07-13 12:53:27.0,NaN,07/2010
2,232586001,V,3405,SC,MOCA,07/2010,1.0,0.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,29.0,07/2010,2010-07-13 16:23:49.0,NaN,07/2010
3,234494501,V,3406,SC,MOCA,07/2010,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,29.0,07/2010,2010-07-22 11:01:44.0,NaN,07/2010
4,226892001,V,3402,SC,MOCA,06/2010,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,30.0,06/2010,2010-06-21 10:51:41.0,NaN,06/2010


### 'Semantic_Fluency.csv'

In [270]:
SFT=dict_files[files_names[11]]
SFT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,VLTANIM,VLTVEG,VLTFRUIT,AGE_ASSESS_SFTANIM,DVS_SFTANIM,DVT_SFTANIM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233165501,V,3400,BL,SFT,07/2010,22,24,17,39.0,11.0,42.0,07/2010,2010-07-15 17:15:37.0,NaN,07/2010
1,232532101,V,3401,BL,SFT,07/2010,12,18,18,56.0,5.0,25.0,07/2010,2010-07-13 13:42:06.0,NaN,07/2010
2,234840401,V,3404,BL,SFT,07/2010,16,14,11,55.0,8.0,41.0,07/2010,2010-07-23 11:49:01.0,NaN,07/2010
3,231742501,V,3403,BL,SFT,07/2010,19,9,10,68.0,10.0,48.0,07/2010,2010-07-09 09:37:11.0,NaN,07/2010
4,236091601,V,3405,BL,SFT,07/2010,25,20,16,63.0,13.0,57.0,07/2010,2010-07-30 10:20:44.0,NaN,07/2010


### 'Geriatric_Depression_Scale__Short_.csv'

In [273]:
GDSSHORT=dict_files[files_names[12]]
GDSSHORT.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,GDSSATIS,GDSDROPD,GDSEMPTY,GDSBORED,...,GDSMEMRY,GDSALIVE,GDSWRTLS,GDSENRGY,GDSHOPLS,GDSBETER,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,234843501,V,3404,BL,GDSSHORT,07/2010,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,07/2010,2010-07-23 11:56:16.0,NaN,07/2010
1,233822201,V,3400,BL,GDSSHORT,07/2010,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,07/2010,2010-07-20 07:21:07.0,NaN,07/2010
2,232534901,V,3401,BL,GDSSHORT,07/2010,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,07/2010,2010-07-13 13:48:13.0,NaN,07/2010
3,231744001,V,3403,BL,GDSSHORT,07/2010,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,07/2010,2010-07-09 09:43:18.0,NaN,07/2010
4,236092601,V,3405,BL,GDSSHORT,07/2010,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,07/2010,2010-07-30 10:28:27.0,NaN,07/2010


### 'Symbol_Digit_Modalities.csv'

In [274]:
SDM=dict_files[files_names[13]]
SDM.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,SDMTOTAL,SDMTVRSN,AGE_ASSESS_SDM,DVSD_SDM,DVT_SDM,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,231742901,V,3403,BL,SDM,07/2010,32.0,1.0,68.0,-1.000,40.000000,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010
1,232533101,V,3401,BL,SDM,07/2010,53.0,1.0,56.0,0.625,56.250000,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010
2,233165901,V,3400,BL,SDM,07/2010,47.0,1.0,39.0,-0.667,43.330002,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010
3,234842101,V,3404,BL,SDM,07/2010,50.0,1.0,55.0,0.250,52.500000,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010
4,236092001,V,3405,BL,SDM,07/2010,53.0,1.0,63.0,0.625,56.250000,NaN,07/2010,2012-06-03 09:09:11.0,NaN,07/2010


### 'State-Trait_Anxiety_Inventory.csv'

In [279]:
STAI=dict_files[files_names[14]]
STAI.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,STAIAD1,STAIAD2,STAIAD3,STAIAD4,...,STAIAD35,STAIAD36,STAIAD37,STAIAD38,STAIAD39,STAIAD40,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,234843901,V,3404,BL,STAI,07/2010,3.0,3.0,1.0,1.0,...,1.0,3.0,2.0,1.0,3.0,2.0,07/2010,2010-07-23 11:58:39.0,NaN,07/2010
1,233826001,V,3400,BL,STAI,07/2010,3.0,2.0,1.0,1.0,...,2.0,3.0,4.0,2.0,3.0,4.0,07/2010,2010-07-20 07:28:30.0,NaN,07/2010
2,232536101,V,3401,BL,STAI,07/2010,4.0,4.0,1.0,1.0,...,1.0,4.0,2.0,1.0,4.0,1.0,07/2010,2010-07-13 13:52:33.0,NaN,07/2010
3,236092801,V,3405,BL,STAI,07/2010,4.0,4.0,1.0,1.0,...,2.0,4.0,2.0,2.0,4.0,1.0,07/2010,2010-07-30 10:30:33.0,NaN,07/2010
4,231746001,V,3403,BL,STAI,07/2010,2.0,2.0,2.0,2.0,...,2.0,3.0,1.0,1.0,3.0,2.0,07/2010,2010-07-09 09:47:13.0,NaN,07/2010


### 'Benton_Judgment_of_Line_Orientation.csv'

In [286]:
BENTONOD=dict_files[files_names[15]]
BENTONOD.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,BJLOT1,BJLOT2,BJLOT3,BJLOT4,...,COMM,JLO_TOTRAW,JLO_TOTCALC,AGE_ASSESS_JLO,DVS_JLO_MSSA,DVS_JLO_MSSAE,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233165301,V,3400,BL,BENTONOD,07/2010,1.0,NaN,1.0,NaN,...,NaN,13.0,26.0,39.0,13.0,11.70,07/2010,2010-07-15 17:15:13.0,NaN,07/2010
1,232531901,V,3401,BL,BENTONOD,07/2010,0.0,NaN,1.0,NaN,...,NaN,9.0,18.0,56.0,8.0,5.97,07/2010,2010-07-13 13:41:35.0,NaN,07/2010
2,234839901,V,3404,BL,BENTONOD,07/2010,1.0,NaN,1.0,NaN,...,NaN,15.0,30.0,55.0,16.0,15.92,07/2010,2010-07-23 11:46:15.0,NaN,07/2010
3,236091401,V,3405,BL,BENTONOD,07/2010,1.0,NaN,1.0,NaN,...,NaN,14.0,28.0,63.0,14.0,12.80,07/2010,2010-07-30 10:20:02.0,NaN,07/2010
4,231742101,V,3403,BL,BENTONOD,07/2010,1.0,NaN,1.0,NaN,...,NaN,13.0,26.0,68.0,13.0,11.70,07/2010,2010-07-09 09:36:16.0,NaN,07/2010


### 'QUIP_Current_Short.csv'

In [283]:
QUIPCS=dict_files[files_names[16]]
QUIPCS.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,PTINBOTH,TMGAMBLE,CNTRLGMB,TMSEX,...,CNTRLEAT,TMTORACT,TMTMTACT,TMTRWD,TMDISMED,CNTRLDSM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,232537101,V,3401,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-13 13:56:29.0,NaN,07/2010
1,233826501,V,3400,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-20 07:29:44.0,NaN,07/2010
2,234844401,V,3404,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-23 11:59:49.0,NaN,07/2010
3,231746401,V,3403,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-09 09:48:15.0,NaN,07/2010
4,236093001,V,3405,BL,QUIPCS,07/2010,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,07/2010,2010-07-30 10:31:15.0,NaN,07/2010


## 1. Data Dictionary

### Reading the data dictionary

In [21]:
# reading the data dictionary for understanding the features
dd_files=glob('data/Data___Databases/*.csv')
dict_ddfiles={}
for files in dd_files:
#for i, files in enumerate(csv_files): 
    df=pd.read_csv(files)
    basepath, filename=ntpath.split(files)
    dict_ddfiles[filename]=df
#dict_ddfiles

In [22]:
# looking at the Data_Dictionary
DATADICT=dict_ddfiles['Data_Dictionary.csv']
dd=DATADICT
print('length:{}'.format(len(dd)))
print('number of features:{}'.format(len(dd.columns)))
l=list(set(dd['MOD_NAME']))
#sorted(l)

length:2982
number of features:11


In [23]:
# Take only the MOD_NAME of Medical
MOD_MED=['CONMED','CURRCOND','DIAGFEAT','INCIDENTS','MHXGEN','GENPHYEX','PDMEDUSE','CLINDX','INITMD','PDFEAT','PREGNANC','SURGPD','MEDPD','VITAL']
dd_med=dd[dd['MOD_NAME'].isin(MOD_MED)]
print('length dd_med:{}'.format(len(dd_med)))
dd_med

length dd_med:358


,MOD_NAME,ITM_NAME,SEQ_NO,DSCR,ITM_TYPE,FLD_LEN,DECML,MIN_LEN,MAX_LEN,CODELIST,update_stamp
159,CLINDX,NaN,0,Clinical Diagnosis and Management,NaN,NaN,NaN,NaN,NaN,NaN,2017-06-12 07:11:48.0
160,CLINDX,REC_ID,1,Record ID,NUMBER,12.0,0.0,NaN,NaN,NaN,2017-06-12 07:11:48.0
161,CLINDX,F_STATUS,2,"Data status: Verified, Secured or Locked",CHAR,1.0,NaN,NaN,NaN,NaN,2017-06-12 07:11:48.0
162,CLINDX,CNO,3,Center Number,CHAR,4.0,NaN,NaN,NaN,NaN,2017-06-12 07:11:48.0
163,CLINDX,PATNO,4,Patient Number,CHAR,8.0,NaN,NaN,NaN,NaN,2017-06-12 07:11:48.0
164,CLINDX,EVENT_ID,5,Event Name,CHAR,8.0,NaN,NaN,NaN,NaN,2017-06-12 07:11:48.0
165,CLINDX,PAG_NAME,6,Page Name,CHAR,8.0,NaN,NaN,NaN,NaN,2017-06-12 07:11:48.0
166,CLINDX,INFODT,7,Event Date,DATE,NaN,NaN,NaN,NaN,NaN,2017-06-12 07:11:48.0
167,CLINDX,PSLVL,8,Confidence level-motor signs of PS,CHAR,1.0,NaN,NaN,NaN,PSLVL,2017-06-12 07:11:48.0
168,CLINDX,DCNOMTR,9,No motor signs consistent with PS,CHAR,1.0,NaN,NaN,NaN,YN,2017-06-12 07:11:48.0


In [24]:
print(len(list(set(dd_med['MOD_NAME']))))
list(set(dd_med['MOD_NAME']))

12


['CURRCOND',
 'SURGPD',
 'PDFEAT',
 'CLINDX',
 'INCIDENTS',
 'MHXGEN',
 'DIAGFEAT',
 'GENPHYEX',
 'CONMED',
 'VITAL',
 'PREGNANC',
 'PDMEDUSE']

In [25]:
# looking at the codes list
CODELIST=dict_ddfiles['Code_List.csv']
CL=CODELIST
print('length:{}'.format(len(CL)))
print('number of features:{}'.format(len(CL.columns)))

CL_med=CL[CL['PAG_NAME'].isin(MOD_MED)]
print('length CL_med:{}'.format(len(CL_med)))
#CL_med

length:2274
number of features:6
length CL_med:264
